## Population Model

In [ ]:
import numpy as np
import pandas

# Adjustables
is_sim = True
drug = 'large_data_1'
num_PK_comp = 1
observation_name = 'Platelets '
noise_mult = 1
data_mixed_params = ["V_c"]  # 
model_mixed_params = ["V_c", "K_cl"]  # 
# Data 1-pop                Profile likelihoods
    # All - 1  2  3  4                  
    # V_c - 1  2  3  4                   
    # K_cl - 1  2  3  4     
    # None - 1  2  3  4     
# Data 2-pop
    # All - 1  2  3  4      
    # V_c - 1  2  3  4      
    # K_cl - 1  2  3  4     
    # None - 1  2  3  4     
model_no_pooled_params = []
model_fixed_params = ["K_1", "V_1", "sigma_PK"]

In [ ]:
# Set folders to save to
import os

if is_sim:
    PK_data_file = "../Data_and_parameters/PK_sim/PK_comp"+str(num_PK_comp)+"/"
    PD_data_file = "../Data_and_parameters/PD_sim/PK_comp"+str(num_PK_comp)+"/"
else:
    PK_data_file = "../Data_and_parameters/PK_real/"
    PD_data_file = "../Data_and_parameters/PD_real/"


# Collect fixed and mixed effects params
PK_params = ["K_cl", "V_c", "K_1", "V_1", "K_2", "V_2",  "sigma_PK"]
PD_params = ["S", "R_0", "Nonsense", "gamma", "MTT", "sigma_PD"]
all_params = (
    [PD_params[0]] + PK_params[:2*num_PK_comp] + PD_params[1:-1] + PK_params[-1:]
    + PD_params[-1:]
)
PK_params = PK_params[:2*num_PK_comp] + PK_params[-1:]

PKPD_param_numbers = dict(zip(all_params, range(11)))
PK_param_numbers = dict(zip(PK_params, range(6)))
PKPD_pop_param_numbers = {}
pos = 0
for param in [x for x in all_params if (x not in model_fixed_params)]:
    PKPD_pop_param_numbers[param] = pos
    pos +=1
    if param in model_mixed_params:
        PKPD_pop_param_numbers["omega_"+param] = pos
        pos +=1

PK_pop_param_numbers = {}
pos = 0
for param in PK_params:
    PK_pop_param_numbers[param] = pos
    pos +=1
    if param in model_mixed_params:
        PK_pop_param_numbers["omega_"+param] = pos
        pos +=1

# File Name additions
fixed_file = ""
model_fixed_params = sorted(model_fixed_params)
if len(model_fixed_params) > 0:
    fixed_file += "_fixed"
    for param in model_fixed_params:
        fixed_file += "_"+param

PK_pop_model_file = ""
if len(model_mixed_params) == 0 and len(model_no_pooled_params) == 0:
    PK_pop_model_file += "fixed_effects"
else:
    PK_pop_model_file += "pop"
    for param in model_mixed_params:
        if param in PK_params:
            PK_pop_model_file += "_"+param
    if len(model_no_pooled_params) > 0:
        PK_pop_model_file += "_independ"
        for param in model_no_pooled_params:
            if param in PK_params:
                PK_pop_model_file += "_"+param

PD_pop_model_file = ""
if len(model_mixed_params) == 0 and len(model_no_pooled_params) == 0:
    PD_pop_model_file += "fixed_effects"
else:
    PD_pop_model_file += "pop"
    for param in model_mixed_params:
        PD_pop_model_file += "_"+param
    if len(model_no_pooled_params) > 0:
        PD_pop_model_file += "_independ"
        for param in model_no_pooled_params:
            PD_pop_model_file += "_"+param

if is_sim:
    if len(data_mixed_params)==0:
        drug += "_no_pop"
    else:
        drug += "_"+str(len(data_mixed_params))+"pop"
    if noise_mult != 1:
        drug += "_small_noise_"+str(noise_mult)


PK_image_file = "../Images/PK_sim/PK_comp"+str(num_PK_comp)+"/"+drug
PD_image_file = "../Images/PD_sim/PD_comp"+str(num_PK_comp)+"/"+drug
mono_file = "../Monolix/PK_sim/PK_comp"+str(num_PK_comp)+"/"+drug

os.makedirs(PK_data_file+drug, exist_ok=True)
os.makedirs(PD_data_file+drug, exist_ok=True)
os.makedirs(PK_image_file, exist_ok=True)
os.makedirs(PD_image_file, exist_ok=True)

### PK Analysis

For the simulated dataset, we need to acquire the data generating parameters. Typical values are the same from the poooled analysis, so we will load those results. The values for $\Omega$ we have to set here.

In [ ]:
def create_PK_true_param():
    PK_comp2_true_typs = np.load("/home/rumney/Documents/Myleotoxicity/Myleotoxicity-PKPD/Data_and_parameters/PK_sim/PK_comp2/actual_params.npy")[1]
    PK_comp2_true_typs = PK_comp2_true_typs.astype(float)

    if num_PK_comp==1:
        PK_comp_vols = [['V_c, L'], [PK_comp2_true_typs[1]+PK_comp2_true_typs[3]]]
    else:
        PK_comp_vols = [
            ['V_c, L']+['V_'+str(x)+', L' for x in range(1, num_PK_comp)],
            [PK_comp2_true_typs[1]]+[PK_comp2_true_typs[3]/num_PK_comp]*num_PK_comp
        ]
    PK_rates = [
        ['K_{cl}, L/hr'] + ['K_'+str(x)+', L/hr' for x in range(1, num_PK_comp)],
        [PK_comp2_true_typs[0]] + [PK_comp2_true_typs[3]/x for x in range(1, num_PK_comp)]
    ]

    PK_true_typs = np.empty((2, 2*num_PK_comp+1), dtype='<U32')
    PK_true_typs[:, 0:-1:2] = PK_rates
    PK_true_typs[:, 1:-1:2] = PK_comp_vols
    PK_true_typs[:, -1] = ['sigma_{m, PK}', PK_comp2_true_typs[-1]]

    # PK_true_typs[1, 0] = 0.5*float(PK_true_typs[1, 0])
    np.save(PK_data_file+"actual_params.npy", PK_true_typs)

    PK_true_omegas = [0.3]*(num_PK_comp+1)
    np.save(PK_data_file+"omega.npy", PK_true_omegas)

    return PK_true_typs, PK_true_omegas

# Set the bounds on the parameters
# bounds = np.asarray([
#     [
#         0.01,
#         0.1*V_c_approx,
#         # 0.01,
#         # 0.01*V_c_approx,
#         0.0001
#     ],
#     [
#         100,
#         10*V_c_approx,
#         # 100,
#         # 100*V_c_approx,
#         1
#     ]
# ])

# np.save(PK_data_file+"bounds.npy", bounds)

In [ ]:
from plotly import figure_factory as ff
from pandas.core.dtypes.common import is_integer

try:
    # Load the Data-generating typical parameters
    PK_true_typs = np.load(PK_data_file+"actual_params.npy")

    # Load the Data-generating inter-individual variability parameters
    PK_true_omegas = np.load(PK_data_file+"omega.npy")
except FileNotFoundError:
    PK_true_typs, PK_true_omegas = create_PK_true_param()

split_param_names = np.char.split(PK_true_typs[0, :], ", ")
PK_true_pops = PK_true_typs
for param in PK_params:
    pos_typ = PK_param_numbers[param]
    pos_pop = PK_pop_param_numbers[param]
    if param in data_mixed_params:
        omega = PK_true_omegas[pos_typ]
    else:
        omega = 1e-5

    if param in model_mixed_params:
        PK_true_pops = np.insert(
            PK_true_pops,
            pos_pop+1,
            ["omega_{"+split_param_names[pos_typ][0]+"}", omega],
            axis=1
        )
        PK_true_pops[0, pos_pop] = str().join(
            [split_param_names[pos_typ][0], "_typ", ", "]
            + list(split_param_names[pos_typ][1:])
        )

PK_true_pops[1, -1] = noise_mult*float(PK_true_pops[1, -1])
# PK_bounds = np.load(PK_data_file+"bounds.npy")

table_df = pandas.DataFrame(
    PK_true_pops.transpose(),
    columns=['Parameter name', 'Data generating value']
)
table_df = table_df.astype({'Data generating value': float})

table_df = table_df.round({'Data generating value': 4})
table_df = table_df.set_index('Parameter name').transpose()
fig = ff.create_table(table_df)
fig.update_layout(
    width=500,
    height=45,
)

pop_param_names = PK_true_pops[0, :]
PK_true_pops = PK_true_pops[1, :].astype('float64')

PK_param_names = PK_true_typs[0, :]
PK_true_typs = PK_true_typs[1, :].astype('float64')

fig.write_image(PK_image_file+"/true_values.svg")
fig.show()

In [ ]:
from Code.PK_model import ChiPKLin
import chi
import logging

# Remove Annoying logging
logger = logging.getLogger()
logger.handlers = []

# Set up the model
PK_model = ChiPKLin(num_comp=num_PK_comp)
PK_model.set_administration(
    compartment='central', amount_var='drug_amount')
PK_model.set_outputs(['central.drug_concentration'])
noise_model = chi.MultiplicativeGaussianErrorModel()
n_noise = noise_model.n_parameters()
chi_param_names = np.concatenate((PK_model.parameters(), noise_model.get_parameter_names()))

pop_models = []
for i_param, param in enumerate(PK_params):
    chi_param = chi_param_names[i_param]
    if param in model_mixed_params:
        print(chi_param, "mixed")
        pop_models.append(chi.LogNormalModel(n_dim=1))
    elif param in model_no_pooled_params:
        print(chi_param, "no pooled")
        pop_models.append(chi.HeterogeneousModel(n_dim=1))
    else:
        print(chi_param, "pooled")
        pop_models.append(chi.PooledModel(n_dim=1))

population_model = chi.ComposedPopulationModel(pop_models)

# Set up the inference problem
problem = chi.ProblemModellingController(PK_model, noise_model)
problem.set_population_model(population_model)


In [ ]:
def create_PK_dataset(dose_amts, n_ids_per_dose, times, dose_time=0):
    n_ids_data = n_ids_per_dose*len(dose_amts)

    # Get log of mixed effect typical parameters
    pop_params = PK_true_pops.copy()
    for param in PK_params:
        if param in model_mixed_params:
            pos = PK_pop_param_numbers[param]
            pop_params[pos] = np.log(pop_params[pos])
    
    # Acquire patient parameters
    individual_parameters = population_model.sample(
        parameters=pop_params,
        n_samples=n_ids_data
    )

    # Set up Dataframe
    df = pandas.DataFrame(columns=[
        "ID", "Time", "Observable", "Value", "Dose group", "Duration", "Dose"
    ])
    # Generate data
    for i_dose, dose in enumerate(dose_amts):
        PK_model.set_dosing_regimen(dose=dose, start=dose_time, period=0)
        for i_ind in range(0, n_ids_per_dose):
            # Simulate model
            pat_param = individual_parameters[i_ind+i_dose, :]
            patient_result = PK_model.simulate(pat_param[:-n_noise], times)
            patient_result = noise_model.sample(
                pat_param[-n_noise:], patient_result[0, :]
            )[:, 0]

            # Format patient data
            patient_data= pandas.DataFrame(columns=[
                "ID", "Time", "Observable", "Value", "Dose group", "Duration", "Dose"
            ])
            patient_id = i_ind+(i_dose*n_ids_per_dose)+1
            patient_data["ID"] = [patient_id]*(len(times)+1)
            patient_data["Time"] = np.concatenate((times, [dose_time]))
            patient_data["Observable"] = (
                ['central.drug_concentration']*len(times)+[None]
            )
            patient_data["Value"] = np.concatenate((patient_result, [None]))
            patient_data["Dose group"] = [dose]*(len(times)+1)
            patient_data["Duration"] = [None]*len(times)+[0.01]
            patient_data["Dose"] = [None]*len(times)+[dose]

            # Join to main dataframe
            df = pandas.concat([df, patient_data])
        df = df.reset_index(drop=True)

    return df, individual_parameters

In [ ]:
if is_sim:
    PK_true_ind = {}
    # Check whether Dataset already exists
    try:
        df = pandas.read_csv(PD_data_file + drug+"/data.csv")
        df.drop(
            df.loc[df['Observable'] == 'circulating.R'].index, inplace=True
        )
        dose_time = (df.loc[df['Dose'].notna()])['Time'].min()
        df['Time'] = df['Time']-dose_time
        df.replace(
            {'PK_central.drug_concentration': 'central.drug_concentration'},
            inplace=True
        )
        for param in data_mixed_params:
            pos = PK_param_numbers[param]
            PK_true_ind[param] = np.load(PD_data_file + drug + "/ind_" + param + ".npy")
    except FileNotFoundError:
        try:
            df = pandas.read_csv(PK_data_file + drug+"/data.csv")
            os.makedirs(mono_file, exist_ok=True)
            df_mono = df.copy()
            df_mono.replace(
                {'central.drug_concentration': 'central_drug_conc'},
                inplace=True
            )
            df_mono.to_csv(mono_file + "/data.csv", index=False)
            for param in data_mixed_params:
                pos = PK_param_numbers[param]
                PK_true_ind[param] = np.load(PK_data_file + drug + "/ind_" + param + ".npy")

        except FileNotFoundError:
            print("creating new dataset")
            # Select options for data
            dose_amts = [1.0, 2.0, 3.0]
            n_ids_per_dose = 15
            n_times_data = 50

            # Create and save simulated data
            times = np.linspace(0.05, 5, n_times_data)
            df, ind_params = create_PK_dataset(
                dose_amts, n_ids_per_dose, times
            )
            df.to_csv(PK_data_file + drug+"/data.csv", index=False)

            os.makedirs(mono_file, exist_ok=True)
            df_mono = df.copy()
            df_mono.replace(
                {'central.drug_concentration': 'central_drug_conc'},
                inplace=True
            )
            df_mono.to_csv(mono_file + "/data.csv", index=False)
            for param in data_mixed_params:
                pos = PK_param_numbers[param]
                np.save(
                    PK_data_file + drug + "/ind_" + param + ".npy",
                    ind_params[:, pos]
                )
                PK_true_ind[param] = ind_params[:, pos]

else:
    # Load in the data
    df = pandas.read_csv(PK_data_file + drug+"/data.csv")
    dose_unit = "mg"

    # Reformat this into Something Chi understands
    df = df.rename(columns={
        'TIME': 'Time', 'OBS': 'Value', 'DOSE': 'Dose group'
    })
    df.replace(
        {drug: 'central.drug_concentration'},
        inplace=True
    )
    dosing_df = df.groupby('ID', as_index=False)['Dose group'].mean()
    dosing_df.columns = ['ID', 'Dose']
    dosing_df['Time'] = [0.0]*len(dosing_df)
    dosing_df['Duration'] = [0.001]*len(dosing_df)

    df = pandas.concat([df, dosing_df], join='outer', ignore_index=True)

df_obs = df.loc[df['Observable'] == 'central.drug_concentration']
df = df.loc[df['ID'].isin(df_obs['ID'].unique())]
n_ids_data = len(df["ID"].unique())

dose_unit = "mg"
problem.set_data(df)

df

Before we begin with the parameter inference we can make some estimations of the parameter values. This ensures we have the correct scale of these parameters and helps us with determining the priors and starting points for the maximum posterior method.

In [ ]:
from scipy.integrate import simpson
from scipy.stats import linregress

# Find the first and second points for each individual
df_dose = np.asarray(df.groupby('ID')['Dose'].first())
df_obs['rank'] = df_obs.sort_values('Time').groupby('ID').cumcount()+1
points_1st = df_obs[df_obs['rank'] == 1].sort_values('ID', ignore_index=True)
points_2nd = df_obs[df_obs['rank'] == 2].sort_values('ID', ignore_index=True)

# Determine y-intercept
y_0 = points_1st['Value'] - points_1st['Time'] * (
    (points_1st['Value'] - points_2nd['Value'])
    / (points_1st['Time'] - points_2nd['Time'])
)
# Estimate V_c
V_c_approx = (df_dose/y_0).mean()

# Determine the AUC from the first to last point,
AUC_0_last = np.empty(len(df["ID"].unique()))
# and the last drug concentration value,
C_last = np.empty(len(df["ID"].unique()))
# and the rate of decay of the drug,
lambda_z = np.empty(len(df["ID"].unique()))

for i, patient in enumerate(df["ID"].unique()):
    y_ind = np.asarray(df_obs.loc[df_obs['ID'] == patient]['Value'])
    x_ind = np.asarray(df_obs.loc[df_obs['ID'] == patient]['Time'])
    AUC_0_last[i] = simpson(y=y_ind, x=x_ind)
    C_last[i] = y_ind[-1]
    lambda_z[i] = linregress(
        x=x_ind[int(0.5*len(x_ind)):],
        y=x_ind[int(0.5*len(x_ind)):]
    ).slope

# Extrapolate AUC_0_last to infinity
AUC_inf = AUC_0_last+C_last/lambda_z
# Approximate the clearance
cl_approx = (df_dose/AUC_inf).mean()

Approximations = [
        cl_approx,          # Clearance
        V_c_approx          # Central volume
    ]+[
        cl_approx,          # Periferal compartment transfer
        V_c_approx          # Periferal compartment volume
    ]*(num_PK_comp-1)+[
        0.1,                # PK Noise parameter
]

table_df.loc["Approximate"] = [np.nan]*len(table_df.columns)
for i, param in enumerate(PK_params):
    pos = PK_pop_param_numbers[param]
    table_df.iat[1, pos] = Approximations[i]
table_df = table_df.round(4)

fig = ff.create_table(table_df, index=True)
fig.update_layout(
    width=500,
    height=65
)
# fig.write_image(PK_image_file+"/data_table.svg")
fig.show()

# Determine the shape parameters for the priors
shape_parameters = [
        0.3,                       # Clearance
        0.3,                        # Central volume
    ]+[
        3,                          # Periferal compartment transfer
        0.6,                        # Periferal compartment volume
    ]*(num_PK_comp-1)+[
        0.4,                        # PK Noise parameter
]

In [ ]:
import pints

# Build the priors and transformation of the parameter space
log_priors = []
transformations = []
for param in PK_params:
    if param in model_mixed_params:
        # Prior for the typical value
        approx = np.log(Approximations[PK_param_numbers[param]])
        shape = shape_parameters[PK_param_numbers[param]]
        log_priors.append(pints.GaussianLogPrior(approx, shape))
        # Prior for omega
        approx = np.log(shape_parameters[PK_param_numbers[param]])
        log_priors.append(pints.LogNormalLogPrior(approx, 0.2))
        # Transformation for the individual parameters
        ind_trans = pints.LogTransformation(n_ids_data)
        transformations = [ind_trans] + transformations
        # Transformation for the population parameters
        transformations.append(pints.IdentityTransformation(2))
    else:
        # Prior and transformation for the pooled parameter
        approx = np.log(Approximations[PK_param_numbers[param]])
        shape = shape_parameters[PK_param_numbers[param]]
        log_priors.append(pints.LogNormalLogPrior(approx, shape))
        transformations.append(pints.LogTransformation(1))

# Compose the priors together
log_prior = pints.ComposedLogPrior(*log_priors)
problem.set_log_prior(log_prior)

# Compose the transformations together
transformation = pints.ComposedTransformation(*transformations)


#### Visualisation

In [ ]:
from Code.Plot import Plot_Models

pop_parameters = PK_true_pops.copy()
plot_ind = {}
for param in model_mixed_params:
    pos = PK_pop_param_numbers[param]
    pop_parameters[pos] = np.log(pop_parameters[pos])
    pos = PK_param_numbers[param]
    if param in data_mixed_params:
        plot_ind[chi_param_names[pos]] = PK_true_ind[param]

plot = Plot_Models(population_model, noise_model, PK_model, log_prior)
plot.set_data(df)
fig = plot.plot_pop_distribution(
    pop_parameters, plot_ind
)
# fig.write_image(
#     image_file + "PK_"+drug+"/"
#     +PD_pop_model_file+"_population_distribution.svg"
# )
fig.show()

In [ ]:
lower = []
upper = []
PK_bounds =  np.load(PK_data_file+"bounds.npy")
i = 0
for i, param in enumerate(PK_params):
    if param in model_mixed_params:
        lower.append(np.log(PK_bounds[0, i]))
        upper.append(np.log(PK_bounds[1, i]))

        lower.append(0)
        upper.append(1.2)
    else:
        lower.append(0)
        upper.append(PK_bounds[1, i])

fig = plot.plot_prior(bounds=(lower, upper))
# fig.write_image(
#     PK_image_file + "/" + PD_pop_model_file + "_hyperpriors.svg"
# )
fig.show()

In [ ]:
plot = Plot_Models(population_model, noise_model, PK_model, log_prior, data=df)
fig = plot.plot_over_time(PK_true_pops, show_data=True, title="Data")
fig.write_image(
    PK_image_file + "/" + PK_pop_model_file + "_data.svg"
)
fig.show()

#### Maximum A Posteriori method

In [ ]:
n_runs = 10
log_posterior = problem.get_log_posterior()

In [ ]:
from Code.Inference import OptimisationController
from plotly import figure_factory as ff

# Set the start point for optimisation
start_point = np.asarray(Approximations)

n_mix = 0
for param in PK_params:
    if param in model_mixed_params:
        pos = PK_pop_param_numbers[param]
        start_point[pos] = np.log(start_point[pos])
        start_point = np.insert(start_point, pos+1, 0.4)
        n_mix +=1


# Optimise the model with respect to the data
optimisation = OptimisationController(
    log_posterior
)
optimisation.set_n_runs(n_runs*10)
initial_params = optimisation._initial_params

i = 0
for ini_param in initial_params:
    print(i, ini_param[n_mix*n_ids_data:])
    if pandas.isna(log_posterior(ini_param)):
        for j, param in enumerate(ini_param):
            test_param = ini_param.copy()
            test_param[j] = 0.5*param
            if not pandas.isna(log_posterior(test_param)):
                print("parameter "+str(j-n_ids_data)+" too large: "+str(param))
            test_param[j] = 1.5*param
            if not pandas.isna(log_posterior(test_param)):
                print("parameter "+str(j-n_ids_data)+" too small: "+str(param))
        initial_params = np.delete(initial_params, i, axis=0)
    else:
        i += 1
        if i >= n_runs-1:
            print("successful")
            break

optimisation.set_n_runs(n_runs)
optimisation.set_initial_point([1], [start_point])
optimisation.set_initial_point(
    range(2, n_runs+1),
    initial_params[:n_runs-1, n_mix*n_ids_data:],
    initial_params[:n_runs-1, :n_mix*n_ids_data]
)

In [ ]:
optimisation.set_optimiser(pints.CMAES)
optimisation.set_transform(transformation)
optimisation.set_parallel_evaluation(True)
result = optimisation.run(show_run_progress_bar=False, log_to_screen=False)

# Show summary of optimisation
print('Log-Posterior Value: \t'+str(result['Score'].unique()))
time = np.asarray(result['Time'].unique())
print('Time Taken: \t'+str((time/60).astype(int))+" minutes, "+str((time%60).astype(int))+" seconds, ")

In [ ]:
opt_pop_params = np.empty((n_runs, len(start_point)))
opt_ind_params = np.empty((
    n_runs, n_ids_data*len(model_mixed_params)
))
column_names = ['Parameter']
for run in range(1, n_runs+1):
    column_names.append('Run ' + str(run))
    run_res = result.loc[result['Run'] == run]
    opt_pop_params[run-1] = run_res.loc[run_res['ID'].isnull()]['Estimate'].values
    opt_ind_params[run-1] = run_res.dropna(subset=['ID'])['Estimate'].values

scores = np.transpose([result.groupby(["Run"])['Score'].first()])
column_names = column_names + ['Mean', 'True']
summary_data = np.concatenate((
    [np.concatenate((pop_param_names, ["Log-posterior"]))],
    np.concatenate((opt_pop_params, scores), axis=1),
    [np.concatenate((np.mean(opt_pop_params, axis=0), [np.NaN]))],
    [np.concatenate((PK_true_pops, [np.NaN]))]
), axis=0)
summary_df = pandas.DataFrame(summary_data.transpose(), columns=column_names)


summary_df.to_csv(
    PK_data_file+drug+'/'+PK_pop_model_file+"_opt_pop.csv", index=False
)
np.save(PK_data_file+drug+'/'+PK_pop_model_file+"_opt_ind.npy", opt_ind_params)

In [ ]:
opt_ind_params = np.load(PK_data_file+drug+'/'+PK_pop_model_file+"_opt_ind.npy")
summary_df = pandas.read_csv(
    PK_data_file+drug+'/'+PK_pop_model_file+"_opt_pop.csv"
)
summary_df = summary_df.set_index("Parameter")

print('Result:')
table_df = summary_df.copy()
table_df = table_df.transpose()
n_runs = len(table_df)-2
for param in model_mixed_params:
    if param in PK_params:
        pos = PK_pop_param_numbers[param]
        col = pop_param_names[pos]
        param_typ = np.asarray(table_df[col])
        param_typ[:-1] = np.exp(param_typ[:-1])
        table_df[col] = param_typ
rounding = dict(zip(table_df.columns, [3]*6))
rounding["Log-posterior"] = 2
table_df = table_df.round(rounding)
fig = ff.create_table(table_df, index=True)
fig.update_layout(
    width=900,
    height=250,
)
# fig.write_image(PK_image_file+'/'+PK_pop_model_file+"_opt_table.svg")
fig.show()

In [ ]:
from Code.Plot import Plot_Models

log_posterior_values = summary_df.loc["Log-posterior"].drop(["Mean", "True"])
log_posterior_values = np.asarray(log_posterior_values)

converged = np.bitwise_and((log_posterior_values>=(np.sort(log_posterior_values)[-2]+np.log(0.02))), (log_posterior_values<=(np.sort(log_posterior_values)[-2]-np.log(0.02))))
summary_reduced = summary_df.copy()
summary_reduced = summary_reduced.iloc[:, :-2]
summary_reduced = summary_reduced.loc[:, np.logical_not(converged)]
mean_of_converged = summary_df.iloc[:, :-2].loc[:, converged].mean(axis=1)
summary_reduced["Mean of converged"] = mean_of_converged
summary_reduced["True"] = summary_df["True"]

table = summary_reduced.copy().transpose()
i_mix = 0
n_mix = len([n for n in model_mixed_params if n in PK_params])
ind_params = np.empty((n_ids_data, len(summary_reduced) - 1 - n_mix))
for param in PK_params:
    pos_pop = PK_pop_param_numbers[param]
    pos_typ =  PK_param_numbers[param]
    if param in model_mixed_params:
        col = pop_param_names[pos_pop]
        param_typ = np.asarray(table[col])
        param_typ[:-1] = np.exp(param_typ[:-1])
        table[col] = param_typ

        ind_params[:, pos_typ] = opt_ind_params[converged, i_mix::n_mix].mean(axis=0)
        i_mix += 1
    else:
        ind_params[:, pos_typ] = summary_reduced["Mean of converged"].iat[pos_pop]

plot_pop_params = {"Mean of converged": table.loc["Mean of converged"]}
plot_ind_params = {"Mean of converged": ind_params}


for i_run, run in enumerate(summary_reduced.columns[:-2]):
    run_pop_params = table.loc[run]
    plot_pop_params["Failed " + run] = run_pop_params[:-1]
    ind_params = np.empty((n_ids_data, len(summary_reduced) - 1 - n_mix))
    i_mix = 0
    for param, pos in PK_param_numbers.items():
        if param in model_mixed_params:
            ind_params[:, pos] = opt_ind_params[np.logical_not(converged), i_mix::n_mix][i_run]
            i_mix += 1
        else:
            ind_params[:, pos] = run_pop_params[pos + i_mix]
    plot_ind_params["Failed " + run] = ind_params

rounding = dict(zip(table.columns, [3]*6))
rounding["Log-posterior"] = 2
table = table.round(rounding)
fig = ff.create_table(table, index=True)
fig.update_layout(
    width=500,
)
fig.write_image(PK_image_file+'/'+PK_pop_model_file+"_opt_table.svg")
fig.show()


plot = Plot_Models(
    pop_model=population_model,
    error_models=noise_model,
    mech_model=PK_model,
    data=df
)

fig = plot.plot_over_time(plot_pop_params, ind_params=plot_ind_params, show_data=True, doses=None, title="MLP Prediction", highlight_first=True)
fig.write_image(PK_image_file+"/"+PK_pop_model_file+"opt_graph.svg")
fig.show()

#### Likelihood over the Parameter Space

To determine potential identifiability problems in optimisation, we need to view the likelihood score over the parameter space around the optimum. To do this initially, I will view 1d & 2d slices of the parameter space, which pass through the optimum.

In [ ]:
log_posterior = problem.get_log_posterior()
log_likelihood = log_posterior.get_log_likelihood()

In [ ]:
opt_ind_params = np.load(PK_data_file+drug+'/'+PK_pop_model_file+"_opt_ind.npy")
summary_df = pandas.read_csv(
    PK_data_file+drug+'/'+PK_pop_model_file+"_opt_pop.csv"
)
summary_df = summary_df.set_index("Parameter")

log_posterior_values = summary_df.loc["Log-posterior"].drop(["Mean", "True"])
log_posterior_values = np.asarray(log_posterior_values)

best = log_posterior_values==(np.max(log_posterior_values))
pop_best_of_converged = np.asarray(summary_df.iloc[:-1, :-2].loc[:, best]).flatten()
ind_best_of_converged = opt_ind_params[best].flatten()

ref_param = np.concatenate((ind_best_of_converged, pop_best_of_converged))

In [ ]:
parameters = np.asarray(ref_param)
bottom_parameters = ind_best_of_converged
top_parameters = pop_best_of_converged

# Broadcast pooled parameters and reshape bottom parameters to
# (n_ids, n_dim)
bottom_parameters = \
    log_likelihood._population_model.compute_individual_parameters(
        parameters=top_parameters,
        eta=bottom_parameters,
        return_eta=True
    )


In [ ]:
# mono_start = np.asarray([
#     [0.683, 1.426, 1, 0.3],
#     [1, 1, 1, 1],
#     [1, 1, 1, 1]
# ])

# mono_params = np.asarray(
#     ['V_c_pop', 'Cl_pop', 'omega_V_c', 'omega_cl', 'sigma_m']
# )
# mono_order = [1, 3, 0, 2, -1]
# # np.save(mono_file+drug+'/'+PK_pop_model_file+"_param_reorder.npy", mono_order)

# mono_order = np.load(mono_file+drug+'/'+PK_pop_model_file+2"_param_reorder.npy")
# print(log_posterior.get_parameter_names(exclude_bottom_level=True))
# print(mono_params[mono_order])
# mono_runs = {}

# mono_runs['All mixed-effects'] = np.asarray([
#     [0.6724021855, 1.4025881404, 0.2091850911, 0.02255928739, 0.09698379049],
#     [0.6719967763, 1.4016972075, 0.213069647, 0.01770235678, 0.0970284138]
# ])[:, mono_order]

# mono_runs['V_c'] = np.asarray([[2.935460741, 0.2217901089, 5.5304756874, 0.2282212677, 0.09230391296],
#     2.9336475614	0.2215255612	5.5241375691	0.2289263105	0.09221970779
#     2.9341357982	0.2213419462	5.5244650673	0.2285846895	0.09226504396
#     2.9332559936	0.2218517503	5.5227745727	0.2292001046	0.09218387021
#     ])[:, mono_order]

mono_runs = np.loadtxt(mono_file+'/MLE_'+PK_pop_model_file+'.csv')

# mono_runs['K_cl mixed-effects'] = np.asarray([
#     [0.6551785047, 1.3809146212, None, 0.1855965116, 0.1402454901],
#     [0.7380118895, 1.5317061146, None, 0.1874982731, 0.1607835969]
# ])[:, mono_order]

# mono_runs['Fixed-effects'] = np.asarray([
#     [1.3129154335, 2.1500067111, None, None, 0.7957449095],
#     [1.3126919626, 2.1496641454, None, None, 0.795645034]
# ])[:, mono_order]

# conv_fail = [1]
# fisher_fail = []
mono_runs

In [ ]:
from itertools import chain, combinations
import plotly.express.colors as pxclrs

# powerset_params = chain.from_iterable(combinations(PK_params[:-1], r) for r in range(len(PK_params[:-1])+1))
# colour_arg = np.flatnonzero([(x == tuple(model_mixed_params)) for x in powerset_params])[0]
colour_arg = int(drug.split("_")[2]) + 4*(len(data_mixed_params)-1) - 1
model_colour = pxclrs.qualitative.Safe[colour_arg]

In [ ]:
from Code.Plot import Plot_Models
import plotly.graph_objects as go
from scipy import interpolate

plot = Plot_Models(
    pop_model=population_model,
    error_models=noise_model,
    mech_model=PK_model,
    data=df
)
plot.set_colour({"base":model_colour})
lower_bounds = []
upper_bounds = []
n_mix = len(model_mixed_params)*n_ids_data

for param in PK_params:
    pos = PK_pop_param_numbers[param]
    shape_pos = PK_param_numbers[param]
    param_value = ref_param[pos+n_mix]
    if param in model_mixed_params:
        omega_value = np.log(ref_param[pos+n_mix+1])
        lower_bounds.append(param_value-shape_parameters[shape_pos])
        lower_bounds.append(np.exp(omega_value-0.4))
        upper_bounds.append(param_value+shape_parameters[shape_pos])
        upper_bounds.append(np.exp(omega_value+0.4))
    else:
        param_value = np.log(param_value)
        lower_bounds.append(np.exp(param_value-shape_parameters[shape_pos]))
        upper_bounds.append(np.exp(param_value+shape_parameters[shape_pos]))

names = pop_param_names

In [ ]:
force_bounds = (False, False)

# Mixed V_c, K_cl:
lower_bounds[1] = 1e-10  # 1: 1e-4  # 2: 1e-4  # 3: 1e-4  # 4: 1e-4
force_bounds=(
    [False, True, False, False, False],
    [False, False, False, False, False]
)

# Mixed V_c:
# lower_bounds[0] =  2.66  # 1: 2.775  # 2: 2.79  # 3: 2.78  # 4: 2.782  # 5: 2.66  # 6: 2.43  # 7: 2.35  # 8: 2.515
# upper_bounds[0] =  2.76  # 1: 2.815  # 2: 2.825  # 3: 2.82  # 4: 2.817  # 5: 2.76  # 6: 2.56  # 7: 2.45  # 8: 2.575
# lower_bounds[1] =  1.45  # 6: 1.45
# upper_bounds[1] =  1.61  # 6: 1.61
# # upper_bounds[2] =   # 3: 0.75  # 6: 0.85  # 8: 0.62
# force_bounds=(
#     [True, True, False, False],
#     [True, True, False, False]
# )

# Mixed K_cl:
# lower_bounds[2] =   # 1: 5.725  # 2: 5.9  # 3: 5.4  # 4: 5.6  # 5: 5.25  # 6: 5  # 7: 4.9  # 8: 5.1
# lower_bounds[1] =   # 1: 0.06  # 2: 0.06  # 3: 0.08  # 4: 0.075
# upper_bounds[2] =   # 1: 5.92  # 2: 6.1  # 3: 5.7  # 4: 5.9  # 5: 5.425  # 6: 5.275  # 7: 5.125  # 8: 5.4
# force_bounds=(
#     [False, False, True, False],
#     [False, False, True, False]
# )

# Fixed:
# lower_bounds[0] =   # 1: 2.82  # 2: 2.83  # 3: 2.85  # 4: 2.82  # 7: 2.775
# upper_bounds[0] =   # 1: 2.91  # 2: 2.905  # 3: 2.945  # 4: 2.925  # 5: 3.1
# upper_bounds[1] =   # 5: 6.65
# force_bounds=(
#     [False, False, False, False],
#     [True, True, False, False]
# )

In [ ]:
# def log_prior_function(ref_param):
#     return log_prior(ref_param[-4:])

fig = plot.plot_param_function(
    log_likelihood, ref_param, profile="maximum", pairwise=False,
    individual_parameters=False, param_names=names, bounds=(lower_bounds, upper_bounds),
    force_bounds=force_bounds, n_evals=100, profile_opts={"method":"PINTS"}
)

opt_score = log_likelihood(ref_param)

min_x = [np.inf]*(len(ref_param)-n_mix)
max_x = [-np.inf]*(len(ref_param)-n_mix)
min_y = [np.inf]*(len(ref_param)-n_mix)
max_y = [-np.inf]*(len(ref_param)-n_mix)
interpolator = [None]*(len(ref_param)-n_mix)
for trace_data in fig.data:
    param_arg = trace_data.xaxis.split("x")[-1]
    if param_arg == "":
        param_arg = 0
    else:
        param_arg = int(param_arg)-1
    if len(trace_data.x)>2:
        min_x[param_arg] = min(trace_data.x)
        max_x[param_arg] = max(trace_data.x)
        
        min_y[param_arg] = min(trace_data.y)
        max_y[param_arg] = max(trace_data.y)

        interpolator[param_arg] = interpolate.interp1d(trace_data.x, trace_data.y)

cmaes_runs = np.transpose(np.asarray(summary_df.iloc[:-1, :-2]))

for cmaes_run, cmaes_inf_param in enumerate(cmaes_runs):
    # if best[cmaes_run]:
    #     continue

    for param_arg, x in enumerate(cmaes_inf_param):
        row = int(param_arg/3)+1
        col = param_arg%3+1

        if min_x[param_arg]<x<max_x[param_arg]:
            cmaes_param_y = interpolator[param_arg](x)
            if cmaes_param_y<-3.5:
                cmaes_param_y = max_y[param_arg]
        else:
            cmaes_param_y = max_y[param_arg]

        fig.add_trace(
            go.Scatter(
                name="Data-Set "+str(colour_arg+1),
                y=[min_y[param_arg], cmaes_param_y],
                x=[x]*2,
                mode='lines',
                line=dict(color=model_colour, width=2),
                showlegend=(param_arg == 0)&(cmaes_run == 0),
                legendgroup="cmaes",
                legendgrouptitle_text="CMA-ES run result"
            ),
            row=row,
            col=col
        )
    fig.update_yaxes(
        range=[min_y, max_y],
        row=row,
        col=col
    )

for mono_run, mono_inf_param in enumerate(mono_runs):
    for param_arg, x in enumerate(mono_inf_param):
        if x is None:
            continue
        row = int(param_arg/3)+1
        col = param_arg%3+1

        param = list(PK_pop_param_numbers.keys())[param_arg]
        if param in model_mixed_params:
            x = np.log(x)
        
        if min_x[param_arg]<x<max_x[param_arg]:
            mono_param_y = interpolator[param_arg](x)
            if mono_param_y<-3.5:
                mono_param_y = max_y[param_arg]
        else:
            mono_param_y = max_y[param_arg]

        fig.add_trace(
            go.Scatter(
                name="Data-Set "+str(colour_arg+1),
                y=[min_y[param_arg], mono_param_y],
                x=[x]*2,
                mode='lines',
                line=dict(color=model_colour, width=2, dash='dot'),
                showlegend=(param_arg == 0)&(mono_run == 0),
                legendgroup="mononlix",
                legendgrouptitle_text="Mononlix run result"
            ),
            row=row,
            col=col
        )

fig.write_image(PK_image_file+'/'+PK_pop_model_file+"_ll_profiles_compare.svg")
fig.show()

# fig = plot.plot_param_function(
#     log_likelihood, ref_param, profile="maximum", pairwise=True,
#     individual_parameters=False, param_names=names, bounds=(lower_bounds, upper_bounds),
#     force_bounds=force_bounds, n_evals=50
# )
# fig.write_image(PK_image_file+"/"+PD_pop_model_file+fixed_file+"_quick_ll_profiles_pair_from_True.svg")
# fig.show()

In [ ]:
import pickle
with open(PK_image_file+'/'+PK_pop_model_file+"_ll_profiles_data.pkl", "wb") as fp:
    pickle.dump(fig.data, fp)  # encode dict into pickle

In [ ]:
import pickle
timer = pints.Timer()
methods = ['L-BFGS-B', 'TNC', 'COBYLA']

if model_mixed_params == ["K_cl"]:
    param_interest = 2
elif model_mixed_params == ["V_c"]:
    param_interest = 0

force_bounds_ind = [False, False]
if type(force_bounds[0]) is not bool:
    force_bounds_ind[0] = (force_bounds[0])[param_interest]
else:
    force_bounds_ind[0] = force_bounds[0]
if type(force_bounds[1]) is not bool:
    force_bounds_ind[1] = (force_bounds[1])[param_interest]
else:
    force_bounds_ind[1] = force_bounds[1]

for method in methods:
    print(method)
    timer.reset()
    fig = plot.plot_ind_profile_ll(
        log_likelihood, param_interest, ref_param,
        param_name=names[param_interest],
        bounds=(lower_bounds[param_interest], upper_bounds[param_interest]),
        force_bounds=force_bounds_ind, n_evals=100, method=method
    )
    fig.write_image(PK_image_file+'/'+PK_pop_model_file+"_"+method+"_ll_profiles_"+all_params[param_interest]+"_ind_view.svg")
    fig.update_layout(width=300, height=750,)
    fig.show()

    time_str = timer.format(timer.time())
    print("Time to complete graph:", time_str)
    result = [fig.data[0].x, fig.data[0].y]
    try:
        with open(PK_image_file+'/'+PK_pop_model_file+"_ll_profiles_methods_trans_ind_data.pkl", "rb") as fp:
            pickle_data = pickle.load(fp)
    except FileNotFoundError:
        pickle_data = {}
    
    pickle_data[method] = {'Time': time_str, 'Result': result}
    with open(PK_image_file+'/'+PK_pop_model_file+"_ll_profiles_methods_trans_ind_data.pkl", "wb") as fp:
        pickle.dump(pickle_data, fp)  # encode dict into pickle
print("Time to complete graph:", time_str)

#### Bayesian Inference

In [ ]:
# Further adjustables
max_runs = 5
method_name = "NUTS"

if method_name == "NUTS":
    num_samples = 3000  # number of wanted final samples
    method = pints.NoUTurnMCMC
elif method_name == "HBMC":
    num_samples = 60000
    method = pints.HaarioBardenetACMC


In [ ]:
from Code.Inference import SamplingController

log_posterior = problem.get_log_posterior()
sampler = SamplingController(log_posterior)
# sampler.set_n_runs(max_runs)

In [ ]:
inferred_pop_params = pandas.read_csv(PK_data_file+drug+'/'+PK_pop_model_file+"_opt_pop.csv")
inferred_pop_params = inferred_pop_params.set_index("Parameter")
opt_ind_params = np.load(PK_data_file+drug+'/'+PK_pop_model_file+"_opt_ind.npy")

log_posterior_values = inferred_pop_params.loc["Log-posterior"].drop(["Mean", "True"])
log_posterior_values = np.asarray(log_posterior_values)

converged = log_posterior_values >= (np.max(log_posterior_values)+np.log(0.02))
n_runs = np.count_nonzero(converged)

if n_runs < 3:
    print(
        "Not enough consistent results from the Maximum"+
        "Likelihood Estimation to start Sampling"
    )
else:
    if n_runs > max_runs:
        n_runs = max_runs
    sampler.set_n_runs(n_runs)
    opt_params = (
        inferred_pop_params.drop(columns=['Mean', 'True']).values[:, converged]
    )[:-1, :n_runs]
    ind_params = (opt_ind_params[converged])[:n_runs]
    sampler.set_initial_point(
        range(1, n_runs+1), opt_params.transpose(), ind_params
    )
    for i, param in enumerate(population_model.get_parameter_names()):
        print(param, opt_params[i])


In [ ]:
run_length = int(num_samples/n_runs)+100
print("running samplers for "+str(run_length)+" iterations")
cov_0 = None
sampler.set_sampler(method)
sampler.set_stop_criterion(
    max_iterations=run_length
)
sampler.set_transform(transformation)
posterior_samples = sampler.run(
    n_iterations=run_length,
    log_to_screen=True
)
posterior_samples.to_netcdf(PK_data_file+drug+"/"+PK_pop_model_file+"_"+method_name+"_samples.nc")

In [ ]:
import xarray as xr
posterior_samples = xr.open_dataset(PK_data_file+drug+"/"+PK_pop_model_file+"_"+method_name+"_samples.nc").load()
posterior_samples

In [ ]:
from Code.Plot import Plot_Models
import matplotlib.pyplot as plt
import arviz as az
plt.rcParams['svg.fonttype'] = 'none'

lines = list(zip(*(population_model.get_parameter_names(),[{}]*len(PK_true_pops), PK_true_pops)))

# Discard warmup iterations
discard = 100 # max(len(posterior_samples.draw)-int(num_samples/n_runs), 1)
main_samples = posterior_samples.drop_isel(draw=range(0, discard))

print("R-hat value:", az.rhat(main_samples))

plot_names = pop_param_names.copy()
for param in model_mixed_params:
    if param in PK_param_numbers:
        pos = PK_param_numbers[param]
        ind_param_name = "Individual " + PK_param_names[pos]
        plot_names = np.concatenate((plot_names, [ind_param_name]))

plot = Plot_Models(population_model, noise_model, PK_model, log_prior, df)
fig = plot.plot_param_sampling(main_samples, lines=lines, param_names=plot_names, legend=True)
fig.savefig(PK_image_file+'/'+PK_pop_model_file+"_"+method_name+"_Pop_trace_plot.svg")
# fig

In [ ]:
posterior_samples.close()

#### Comparison of Model assumptions

In [ ]:
PK_image_file = "../Images/PK_sim/PK_comp"+str(num_PK_comp)+"/"

In [ ]:
from plotly import figure_factory as ff

number_of_converged = [  # All, V_C, K_cl, None
    ["1 (1 M-E param)","10", "10", "10", "10"],
    ["2 (1 M-E param)","10", "10", "10", "10"],
    ["3 (1 M-E param)","10", "10", "10", "10"],
    ["4 (1 M-E param)","10", "10", "10", "10"],
    ["5 (2 M-E params)","10", "10", "10", "10"],
    ["6 (2 M-E params)","10", "6, 3, 1", "10", "10"],
    ["7 (2 M-E params)","10", "1, 8, 1", "10", "10"],
    ["8 (2 M-E params)","10", "10", "10", "10"],
]

table_df = pandas.DataFrame(data=number_of_converged, columns=["Dataset", "All params M-E", "V_c M-E", "K_{cl} M-E", "F-E"])
# table_df = table_df.set_index("Dataset", )

fig = ff.create_table(table_df)
fig.update_layout(
    width=550,
    height=200
)
fig.write_image(PK_image_file+"converged_comparison.svg")
fig.show()

In [ ]:
import pickle
from scipy import interpolate

fig_range = [-4, 0.02]

print("Mixed-effects:", model_mixed_params)
fig_data = []
for data_set in range(0, 4):
    dataset_file_name = "large_data_"+str(data_set+1)+"_1pop"
    with open(PK_image_file+dataset_file_name+'/'+PK_pop_model_file +"_ll_profiles_data.pkl", 'rb') as fp:
        fig_data += pickle.load(fp)

for data_set in range(0, 4):
    dataset_file_name = "large_data_"+str(data_set+1)+"_2pop"
    with open(PK_image_file+dataset_file_name+'/'+PK_pop_model_file +"_ll_profiles_data.pkl", 'rb') as fp:
        data = pickle.load(fp)
        for trace_data in data:
            if trace_data.xaxis == 'x':
                param_num = 1
            else:
                param_num = int(trace_data.xaxis[-1])
            trace_data.yaxis ='y' + str(len(pop_param_names)+param_num)
            trace_data.xaxis ='x' + str(len(pop_param_names)+param_num)

        fig_data += data

max_y = -np.inf
mix_nums = np.array([])
min_x = np.ones(len(pop_param_names)*2)*(np.infty)
max_x = np.ones(len(pop_param_names)*2)*(-np.infty)
mins = [[]]*5
maxs = [[]]*5
for trace_data in fig_data:
    if trace_data.xaxis == 'x':
        i_axis = 1
    else:
        i_axis = int(trace_data.xaxis[-1])
    
    if i_axis>len(pop_param_names):
        param_num = i_axis-len(pop_param_names)-1
    else:
        param_num = i_axis-1
    
    if (param_num not in (mix_nums+1)) and (param_num != len(pop_param_names)-1):
        n_mix = np.count_nonzero(mix_nums<param_num)
        param = PK_params[param_num-n_mix]
        if param in model_mixed_params:
            trace_data.x = np.exp(trace_data.x)
            mix_nums = np.union1d(mix_nums, [param_num])

    if len(trace_data.x)>2:
        optima_arg = np.argmax(trace_data.y)
        max_y = max(trace_data.y[optima_arg], max_y)

        if trace_data.x[0]<fig_range[0]:
            interpolator = interpolate.interp1d(
                trace_data.y[:optima_arg],
                trace_data.x[:optima_arg]
            )
            x_at_min = interpolator(fig_range[0])
        else:
            x_at_min = trace_data.x[0]
        
        if trace_data.x[-1]<fig_range[0]:
            interpolator = interpolate.interp1d(
                trace_data.y[len(trace_data.y):optima_arg:-1],
                trace_data.x[len(trace_data.y):optima_arg:-1]
            )
            x_at_max = interpolator(fig_range[0])
        else:
            x_at_max = trace_data.x[-1]
        
        min_x[i_axis-1] = min(x_at_min, min_x[i_axis-1])
        max_x[i_axis-1] = max(x_at_max, max_x[i_axis-1])


In [ ]:
import plotly.graph_objects as go

share_xaxes = True

n_cols = len(pop_param_names)
n_rows = 2

fig = go.Figure(data=fig_data).set_subplots(
    n_rows, n_cols, 
    shared_xaxes=share_xaxes, shared_yaxes=True,
    subplot_titles=pop_param_names
)
for col in range(1, len(pop_param_names)+1):
    for row in range(1, 3):
        i_axis = (row-1)*len(pop_param_names) + col

        if share_xaxes:
            CI_x = [
                min(min_x[col-1], min_x[col+len(pop_param_names)-1]),
                max(max_x[col-1], max_x[col+len(pop_param_names)-1])
            ]
        else:
            CI_x = [min_x[i_axis-1], max_x[i_axis-1]]

        fig.add_trace(
            go.Scatter(
                name="Confidence Interval",
                y=[-1.98]*2,
                x=CI_x,
                mode='lines',
                line=dict(color="Black", width=2, dash='dash'),
                showlegend=(i_axis == 1)
            ),
            row=row,
            col=col
        )
        true_param = PK_true_pops[col-1]
        if CI_x[0]<true_param<CI_x[1]:
            fig.add_trace(
                go.Scatter(
                    name="Data-generating parameter",
                    y=fig_range,
                    x=[true_param]*2,
                    mode='lines',
                    line=dict(color="black", width=2),
                    showlegend=(i_axis == 1),
                ),
                row=row,
                col=col
            )

fig.update_yaxes(
    range=fig_range,
    title_text='Log-Likelihood Score'
)
fig.update_xaxes(title_text="Parameter Value")

if len(model_mixed_params)==0:
    title = "Fixed effects model"
else:
    title = "Mixed-effects: "+ ", ".join(model_mixed_params)

fig.update_layout(
    title_text=title,
    width=500+300*n_cols,
    height=250+200*n_rows,
)

if share_xaxes:
    fig.write_image(PK_image_file+ '/' + PK_pop_model_file + "_ll_comparison.svg")
else:
    fig.write_image(PK_image_file+ '/' + PK_pop_model_file + "_ll_comparison_sepx.svg")

fig.show()

### PD Analysis

In [ ]:
# # Load in the data simulated from the Naive model
# df = pandas.read_csv("../Data_and_parameters/PK_sim/sythesised_data_real_timepoints.csv")
# df = df.sort_values(['ID', 'TIME'], ascending=True, ignore_index=True)
# dose_unit = "mg"

# # Reformat this into Something Chi understands
# df = df.rename(columns={'TIME':'Time', 'OBS': 'Value', 'DOSE':'Dose group'})
# df['Observable'] = ['central.drug_concentration']*len(df)
# dosing_df = df.groupby('ID',as_index=False)['Dose group'].mean()
# dosing_df.columns = ['ID', 'Dose']
# dosing_df['Time'] = [0.0]*len(dosing_df)
# dosing_df['Duration'] = [0.001]*len(dosing_df)

# df = pandas.concat([df, dosing_df], join='outer', ignore_index=True)
# df

In [ ]:
# Define the Data-generating parameters
import plotly.figure_factory as ff

PK_true_typs = np.load(PK_data_file+"actual_params.npy")
# PK_actual_pop_params = np.load(PK_data_file+"actual_pop_params.npy")
# print("PK_actual_pop_params", PK_actual_pop_params)
PD_actual_params = np.load(PD_data_file+"actual_params.npy")

# Define population parameters
PK_true_omegas = [1e-10, 1e-10, 0.3] + [1e-10]*8
order = [5, 0, 1, 2, 3, 6, 7, 8,  4, 9]
np.save(PD_data_file+"omega.npy", PK_true_omegas)
PKPD_real_params = np.concatenate((PK_true_typs, PD_actual_params), axis=1)[:, order]
split_param_names = np.char.split(PKPD_real_params[0, :], ", ")
PKPD_real_pop_params = PKPD_real_params
for x in model_mixed_params.values():
    pos = x+np.count_nonzero(np.asarray(list(model_mixed_params.values()))<x)
    PKPD_real_pop_params = np.insert(
        PKPD_real_pop_params,
        pos+1,
        ["omega_"+split_param_names[x][0] , PK_true_omegas[x]],
        axis=1
    )
    PKPD_real_pop_params[0, pos] = str().join([split_param_names[x][0], "_typ", ", "]+list(split_param_names[x][1:]))
PKPD_real_pop_params = np.insert(
    PKPD_real_pop_params,
    [6+np.count_nonzero(np.asarray(list(model_mixed_params.values()))<=6)],
    [['Nonsense'] , [0]],
    axis=1
)

PK_bounds = np.load("../Data_and_parameters/PK_sim/bounds.npy")
PD_bounds = np.load("../Data_and_parameters/PD_sim/bounds.npy")
PKPD_bounds = np.concatenate((PK_bounds , PD_bounds ), axis=1)[:, order]


table_df = pandas.DataFrame(PKPD_real_pop_params.transpose(), columns=['Parameter name', 'Data generating value'])
table_df = table_df.astype({'Data generating value':float})
table_df = table_df.round({'Data generating value':4})
table_df = table_df.set_index('Parameter name').transpose()

fig =  ff.create_table(table_df)
fig.update_layout(
    width=700,
    height=45,
)
# fig.write_image(PD_image_file+"/data_table.svg")

pop_param_names = PKPD_real_pop_params[0, :]
PKPD_real_pop_params = PKPD_real_pop_params[1, :].astype('float64')

PK_param_names = PK_true_typs[0, :]
PD_param_names = PD_actual_params[0, :]

PK_true_typs = PK_true_typs[1, :].astype('float64')
PD_actual_params = PD_actual_params[1, :].astype('float64')
fig.show()


In [ ]:
from Code.PD_model import ChiMyelotoxicityPKPD
from Code.PK_model import ChiPKLin
import chi
import logging

# Remove Annoying logging
logger = logging.getLogger()
logger.handlers = []

# Set up the model
PK_model = ChiPKLin(num_comp=num_PK_comp)
PKPD_model = ChiMyelotoxicityPKPD(PK_model, 'central.drug_concentration')

PKPD_model.set_administration(
    compartment='PK_central', amount_var='drug_amount')

PKPD_model.set_outputs(['PK_central.drug_concentration', 'circulating.R'])

PK_noise_model = chi.MultiplicativeGaussianErrorModel()
PD_noise_model = chi.GaussianErrorModel()

PK_opt_params = pandas.read_csv(PK_data_file+drug+"/"+PK_pop_model_file+"_opt_pop.csv")
PK_opt_params = PK_opt_params.set_index("Parameter")
log_posterior_values = PK_opt_params.loc["Log-posterior"].drop(["Mean", "True"])
log_posterior_values = np.asarray(log_posterior_values)
converged = log_posterior_values>=(np.max(log_posterior_values)+np.log(0.02))
n_conv = np.count_nonzero(converged)

if n_conv<3:
    print("Not enough consistent results from the PK Maximum Likelihood Estimation to start PD")
else:
    PK_opt_params = (PK_opt_params.drop(columns=['Mean', 'True']).values[:, converged])[:-1]
    PK_opt_params = PK_opt_params.mean(axis=1)
print(PK_opt_params)

fixed = {}
pop_models = []
for param in all_params:
    if param == "Nonsense":
        fixed["drug.drug_concentration"]= 0
    elif param in model_fixed_params:
        print(param, "fixed")
        if param == "sigma_PK":
            fixed["PK_central.drug_concentration Sigma rel."] = PK_opt_params[-1]
        else:
            mix_arr = np.asarray(list(model_mixed_params.values()))
            pos = model_fixed_params[param][0] + np.count_nonzero(
                (0 < mix_arr) &
                (mix_arr < model_fixed_params[param][1])
            )
            fixed['PK_'+PK_model.parameters()[model_fixed_params[param][0]]] = PK_opt_params[pos]
    elif param in model_mixed_params:
        print(param, "mixed")
        pop_models.append(chi.LogNormalModel(n_dim=1))
    elif param in model_no_pooled_params:
        print(param, "no pooled")
        pop_models.append(chi.HeterogeneousModel(n_dim=1))
    else:
        print(param, "pooled")
        pop_models.append(chi.PooledModel(n_dim=1))

population_model = chi.ComposedPopulationModel(pop_models)

# Set up the inference problem
problem = chi.ProblemModellingController(PKPD_model, [PK_noise_model, PD_noise_model])
problem.fix_parameters(fixed)
problem.set_population_model(population_model)

# # Set the bounds on the parameters and start point
# # lower_bound = [0.01, 0.1*V_c_approx, 0.01, 0.01*V_c_approx, 0.0001]
# # upper_bound = [100, 10*V_c_approx, 100, 100*V_c_approx, 1]

# # np.save("../Data_and_parameters/PK_sim/bounds", np.asarray([lower_bound, upper_bound]))
# PK_bounds = np.load("../Data_and_parameters/PK_sim/bounds.npy")
# PD_bounds = np.load("../Data_and_parameters/PD_sim/bounds.npy")

# # PD_bounds[0] = 1.2*PD_bounds[0]
# # PD_bounds[1] = 0.6*PD_bounds[1]


In [ ]:
print(fixed)
population_model.get_parameter_names()

In [ ]:
# Create simulated data
dose_amts = [1.0, 2.0, 3.0]
n_ids_data = 15*len(dose_amts)
dose_time = 48
n_times_data = 50

# PK_times = np.linspace(0.05, 5, n_times_data)[:] + dose_time
# PD_times = np.linspace(-dose_time, 500, n_times_data)[:] + dose_time
# data = pandas.DataFrame(columns=["ID", "Time", "Observable", "Value", "Dose group", "Duration", "Dose"])

# # Acquire patient parameters
# reduced_pop_params = PKPD_actual_pop_params.copy()
# reduced_pop_params[list(model_mixed_params.values())] = np.log(reduced_pop_params[list(model_mixed_params.values())])
# reduced_pop_params = np.delete(
#     reduced_pop_params,
#     [x[1]+np.count_nonzero(np.asarray(list(model_mixed_params.values()))<x[1]) 
#      for x in model_fixed_params.values()]
# )
# individual_parameters = population_model.sample(
#     parameters=reduced_pop_params,
#     n_samples=n_ids_data
# )
# # for i, var in enumerate(PKPD_model._const_names):
# #     print(var, individual_parameters[0, i])

# patient = 0
# for i, dose in enumerate(dose_amts):
#     # Set administration and dosing regimen
#     PKPD_model.set_dosing_regimen(dose=dose, start=dose_time, period=0)
#     for _ in range(0, int(n_ids_data/len(dose_amts))):
#         # Simulate model
#         param = np.insert(individual_parameters[patient, :], 6, 0)
#         patient_result = PKPD_model.simulate(param[:-2], np.concatenate((PK_times, PD_times[5:])))
        
#         # produce data
#         include_PK_data = dose!=0.0

#         patient_data = pandas.DataFrame()
#         patient_data_length = len(PK_times)*include_PK_data+len(PD_times)+1
#         patient_data["ID"] = [patient+1]*patient_data_length
#         if include_PK_data:
#             patient_data["Time"] = np.concatenate((PK_times, PD_times, [dose_time]))
#             patient_data["Observable"] = (
#                 ['PK_central.drug_concentration']*len(PK_times)+['circulating.R']*len(PD_times)+[None]
#             )
#             PK_result = PK_noise_model.sample(param[-2:-1], patient_result[0, :len(PK_times)])[:, 0]
#         else:
#             patient_data["Time"] = np.concatenate((PD_times, [dose_time]))
#             patient_data["Observable"] = (
#                 ['circulating.R']*len(PD_times)+[None]
#             )
#             PK_result = []

#         PD_result = PD_noise_model.sample(
#             param[-1:],
#             np.concatenate(([param[5]]*5, patient_result[1, len(PK_times):]))
#         )[:, 0]
#         patient_data["Value"] = np.concatenate((PK_result, PD_result, [None]))

#         patient_data["Dose group"] = [dose]*patient_data_length
#         patient_data["Duration"] = [None]*(patient_data_length-1)+[0.01]
#         patient_data["Dose"] = [None]*(patient_data_length-1)+[dose]
#         data = pandas.concat([data, patient_data])
#         patient += 1

# data.to_csv(PD_data_file + drug+"/data.csv", index=False)
# np.save(PD_data_file + drug+"/ind_Vc_param.npy", individual_parameters[:, 2])

df = pandas.read_csv(PD_data_file + drug+"/data.csv")
n_ids_data = len(df["ID"].unique())

ind_real_params = np.asarray([PKPD_real_pop_params]*n_ids_data)
ind_real_params[:, 2] = np.load(PD_data_file + drug+"/ind_Vc_param.npy")
dose_unit = "mg"

problem.set_data(df)
df

In [ ]:
from scipy.integrate import simpson
from scipy.stats import linregress

# First estimate the parameter V_c. We can do this by drawing a line through the first 2 points and seeing where it crosses the y-axis
df_obs = df.loc[df['Observable']=='PK_central.drug_concentration']
PK_data = df.loc[df['ID'].isin(df_obs['ID'].unique())]
df_dose = np.asarray(PK_data.groupby('ID')['Dose'].first())
df_obs['rank'] = df_obs.sort_values('Time').groupby('ID').cumcount()+1
points_1st = df_obs[df_obs['rank'] == 1].sort_values('ID', ignore_index=True)
points_2nd = df_obs[df_obs['rank'] == 2].sort_values('ID', ignore_index=True)

y_0 = points_1st['Value'] - (points_1st['Time']-dose_time) * (
    (points_1st['Value'] - points_2nd['Value'])
    / ((points_1st['Time']-dose_time)- (points_2nd['Time']-dose_time))
)
V_c_approx = (df_dose/y_0).mean()

AUC_0_last = np.empty(len(PK_data["ID"].unique()))
C_last = np.empty(len(PK_data["ID"].unique()))
lambda_z = np.empty(len(PK_data["ID"].unique()))
for i, patient in enumerate(PK_data["ID"].unique()):
    y_ind = np.asarray(df_obs.loc[df_obs['ID']==patient]['Value'])
    x_ind = np.asarray(df_obs.loc[df_obs['ID']==patient]['Time'])-dose_time
    AUC_0_last[i] = simpson(y=y_ind, x=x_ind)
    C_last[i] = y_ind[-1]
    lambda_z[i] = linregress(x=x_ind[int(0.5*len(x_ind)):], y=x_ind[int(0.5*len(x_ind)):]).slope
AUC_inf = AUC_0_last+C_last/lambda_z
cl_approx = (df_dose/AUC_inf).mean()

df_R_before_dose = df[(df["Time"] < dose_time) & (df["Observable"]=='circulating.R')]
R_0_approx = np.mean(df_R_before_dose["Value"])

nadir = np.empty((2, len(PK_data["ID"].unique())))
for i, patient in enumerate(df["ID"].unique()):
    df_PD_ind = df[(df["ID"]==patient) & (df["Observable"]=='circulating.R')]
    df_nadir = df_PD_ind[df_PD_ind.Value == df_PD_ind.Value.min()]
    y_nadir = df_nadir.Value.iat[0]
    t_nadir = df_nadir.Time.iat[0]-dose_time
    nadir[:, i] = (t_nadir, y_nadir)

MTT_approx = 0.5*nadir[0,:].mean()
S_approx = np.mean((1-(nadir[1,:] - R_0_approx)/(nadir[0,:])*MTT_approx/4)*(V_c_approx/df_dose))

Approximations = [
    S_approx,           # S
    cl_approx,          # Clearance
    V_c_approx,         # Central volume
    cl_approx,          # Periferal compartment transfer
    V_c_approx,         # Periferal compartment volume
    R_0_approx,         # R_0
    0,                  # Nonsense
    0.5,                # gamma
    MTT_approx,         # MTT
    0.1,                # PK Noise parameter
    0.1*R_0_approx,     # PD Noise parameter
]

if len(model_mixed_params) == 0:
    table_df.loc["Approximate"] = np.asarray(Approximations).round(4)
else:
    table_df.loc["Approximate"] = np.insert(
        Approximations,
        np.asarray(list(model_mixed_params.values()))+1,
        [np.nan]*len(model_mixed_params)
    ).round(4)

shape_parameters = [
    0.5,                        # S
    0.75,                       # Clearance
    0.4,                        # Central volume
    3,                          # Periferal compartment transfer
    0.8,                        # Periferal compartment volume
    0.5,                        # R_0
    np.NaN,                     # Nonsense
    1.2,                        # gamma
    0.5,                        # MTT
    0.4,                        # PK Noise parameter
    2.1,                        # PD Noise parameter
]

fig =  ff.create_table(table_df, index=True)
fig.update_layout(
    width=1000,
    height=65
)
# fig.write_image(PD_image_file+"/data_table.svg")
fig.show()

In [ ]:
import pints

log_priors = []
transformations = []
for param in all_params:
    if param in model_mixed_params:
        log_priors.append(pints.GaussianLogPrior(np.log(Approximations[PKPD_param_numbers[param]]), shape_parameters[PKPD_param_numbers[param]]))
        log_priors.append(pints.LogNormalLogPrior(np.log(shape_parameters[PKPD_param_numbers[param]]), 0.4))
        transformations = [pints.LogTransformation(n_ids_data)] +  transformations
        transformations.append(pints.IdentityTransformation(2))
    elif param not in model_fixed_params:
        log_priors.append(pints.LogNormalLogPrior(np.log(Approximations[PKPD_param_numbers[param]]), shape_parameters[PKPD_param_numbers[param]]))
        transformations.append(pints.LogTransformation(1))
        

# log_priors = [
#     pints.UniformLogPrior(PD_bounds[0,0], PD_bounds[1,0]),                  # S
#     pints.LogNormalLogPrior(np.log(cl_approx), 0.75),                       # Clearance
#     pints.GaussianLogPrior(np.log(V_c_approx), 0.4),                        # Log mean of central volume
#     pints.LogNormalLogPrior(-1, 0.4),                                       # Log std. of central volume
#     pints.LogNormalLogPrior(np.log(cl_approx), 3),                          # Periferal compartment transfer
#     pints.LogNormalLogPrior(np.log(V_c_approx), 0.8),                       # Periferal compartment volume
#     pints.LogNormalLogPrior(np.log(R_0_approx), 0.5),                       # R_0
#     pints.LogNormalLogPrior(np.log(0.5), np.sqrt(-np.log(0.25))),           # gamma
#     pints.LogNormalLogPrior(np.log(MTT_approx), 0.5),                       # MTT
#     pints.LogNormalLogPrior(-1, 0.4),                                       # PK Noise parameter
#     pints.LogNormalLogPrior(np.log(0.1*R_0_approx), np.sqrt(-np.log(0.01))),# PD Noise parameter
# ]
# log_priors = 

log_prior = pints.ComposedLogPrior(*log_priors)

# Transform Parameter Space
transformation = pints.ComposedTransformation(*transformations)
#     pints.LogTransformation(len(df['ID'].unique())+2), # Individual parameters, S, Clearance
#     pints.IdentityTransformation(1),                   # Log mean of central volume
#     pints.IdentityTransformation(1),                   # Log std. of central volume
#     pints.LogTransformation(4),                        # Periferal compartment, R_0, gamma, MTT, PK Noise parameter, PD Noise parameter
# )
problem.set_log_prior(log_prior)


<!-- import numpy as np
import pints
import chi
from Code.PK_model import ChiPKLin
import logging

# Remove Annoying logging
logger = logging.getLogger()
logger.handlers = []

# Define parameters
PK_params = np.load("../Data_and_parameters/PK_sim/actual_params.npy")
PK_params = PK_params[:, [1,0,3,2, -1]]
PK_param_names = PK_params[0, :]
PK_params = PK_params[1, :].astype('float64')

# Define population parameters
population_parameters = np.concatenate(([PK_params[0], np.log(PK_params[1]), 0.3], PK_params[2:]))

# Define pharmacokinetic model
mechanistic_model = ChiPKLin(num_comp=2)
mechanistic_model.set_administration(
    compartment='central', amount_var='drug_amount')
error_model = chi.MultiplicativeGaussianErrorModel()

# Define population model

# pop_theta_names = PK_param_names[1].split(",")
# pop_typ_name = pop_theta_names[0]+"_typ,"+pop_theta_names[1]
# pop_omega_name = "omega_"+pop_theta_names[0]

population_model = chi.ComposedPopulationModel([
    chi.PooledModel(n_dim=1, dim_names=[PK_param_names[0]]),
    chi.LogNormalModel(
        n_dim=1, dim_names=[PK_param_names[1]]
    ),
    chi.PooledModel(n_dim=3, dim_names=list(PK_param_names[2:]))])
pop_predictive_model = chi.PopulationPredictiveModel(
    chi.PredictiveModel(mechanistic_model, error_model), population_model)

# Get individual parameters
dose_amts = [1, 2, 4]
n_ids = 12 * len(dose_amts)
individual_parameters = population_model.sample(
    parameters=population_parameters,
    n_samples=n_ids,
    seed=1
)

times = np.arange(0.1, 5.1, 0.5)
data = pandas.DataFrame(columns=["ID", "Time", "Observable", "Value", "Duration", "Dose"])

for i, dose in enumerate(dose_amts):
    # Set administration and dosing regimen
    pop_predictive_model.set_dosing_regimen(dose=dose, period=0)
    patient_data = pop_predictive_model.sample(
        population_parameters, times, include_regimen=True, n_samples=12)
    patient_data["ID"] = patient_data["ID"] + 12*i
    data = pandas.concat([data, patient_data])

problem = chi.ProblemModellingController(mechanistic_model, error_model)
problem.set_population_model(population_model)
problem.set_data(data)
data -->

#### Visualisation

In [ ]:
from Code.Plot import Plot_Models

ind_param_map = {}
pop_parameters = PKPD_real_pop_params.copy()
adj = 0
for param, pos in PKPD_param_numbers.items():
    if param in model_mixed_params:
        ind_param_map[PKPD_model.parameters()[pos]] = ind_real_params[:, pos]
        pop_parameters[pos+adj] = np.log(pop_parameters[pos+adj])
        adj += 1
    elif param in model_fixed_params:
        pop_parameters = np.delete(pop_parameters, pos+adj)
        adj -= 1

plot = Plot_Models(population_model, mech_model=PKPD_model, prior_model=log_prior)
plot.set_data(df)
fig = plot.plot_pop_distribution(
    pop_parameters, ind_param_map
)
fig.write_image(
    PD_image_file+"/"
    +PD_pop_model_file + fixed_file + "_pop_dist.svg"
)
fig.show()


In [ ]:
lower = []
upper = []
param_names = []
i_mix = 0
i_non = 0
for param, pos in PKPD_param_numbers.items():
    if param in model_mixed_params:
        lower.append(np.log(PKPD_bounds[0, pos-i_non]))
        upper.append(np.log(PKPD_bounds[1, pos-i_non]))

        lower.append(0)
        upper.append(1.2)
        param_names.append(pop_param_names[pos+i_mix])
        i_mix += 1
        param_names.append(pop_param_names[pos+i_mix])
    elif param not in model_fixed_params:
        lower.append(0)
        upper.append(PKPD_bounds[1, pos-i_non])
        param_names.append(pop_param_names[pos+i_mix])
    elif param == "Nonsense":
        i_non=1

fig = plot.plot_prior(bounds=(lower, upper), param_names=param_names)
fig.write_image(
    PD_image_file + "/"
    + PD_pop_model_file + fixed_file + "_hyperpriors.svg"
)
fig.show()

In [ ]:
full_pop_models = pop_models.copy()
for param, x in model_fixed_params.items():
    mix_arr = np.asarray(list(model_mixed_params.values()))
    pos = x[1]+ np.count_nonzero(mix_arr < x[1])
    full_pop_models.insert(pos, chi.PooledModel(n_dim=1))

# plot_params = np.insert(PKPD_real_pop_params, 6, 0)
plot = Plot_Models(chi.ComposedPopulationModel(full_pop_models), [PK_noise_model, PD_noise_model], PKPD_model, log_prior, data=df)
fig = plot.plot_over_time(PKPD_real_pop_params, show_data=True, title="PK Data", PK_PD=0)
fig.show()

fig = plot.plot_over_time(PKPD_real_pop_params, show_data=True, title="PD Data", PK_PD=1)
fig.show()

#### Maximum Log likelihood method

In [ ]:
n_runs = 10
log_posterior = problem.get_log_posterior()

In [ ]:
from Code.Inference import OptimisationController
from plotly import figure_factory as ff

# Set the start point for optimisation
start_point = np.asarray(Approximations)
start_point[list(model_mixed_params.values())] = np.log(start_point[list(model_mixed_params.values())])

if len(model_mixed_params) != 0:
    start_point = np.insert(start_point, np.asarray(list(model_mixed_params.values()))+1, 0.4)
start_point = np.delete(
    start_point,
    [x[1]+np.count_nonzero(np.asarray(list(model_mixed_params.values()))<x[1]) 
     for x in model_fixed_params.values()]
)
# PD_start = np.exp(0.5*np.log(PD_bounds[0])+0.5*np.log(PD_bounds[1]))
# PK_start = np.asarray(pandas.read_csv(PK_data_file + drug+"/opt_pop_results.csv")['Optimised'])
# start_point = np.concatenate((PK_start, PD_start[:-2]), axis=0)[order]
# start_point[6] = R_0_approx # Approximation of R_0

# Optimise the model with respect to the data
optimisation = OptimisationController(
    log_posterior
)
optimisation.set_n_runs(n_runs*10)
initial_params = optimisation._initial_params

i=0
for k, ini_param in enumerate(initial_params):
    if pandas.isna(log_posterior(ini_param)):
        print(i, ini_param[n_ids_data:])
        for j, param in enumerate(ini_param):
            test_param = ini_param.copy()
            test_param[j] = 0.5*param
            if not pandas.isna(log_posterior(test_param)):
                print("parameter "+str(j-len(df["ID"].unique()))+" too large: "+str(param))
            test_param[j] = 1.5*param
            if not pandas.isna(log_posterior(test_param)):
                print("parameter "+str(j-len(df["ID"].unique()))+" too small: "+str(param))
        initial_params = np.delete(initial_params, k, axis=0)
    else:
        i+=1
        if i >= n_runs-1:
            print("successful")
            break


# loop_number = 0
# while i<n_runs:
#     ini_param =optimisation._initial_params[i]
#     if pandas.isna(log_posterior(ini_param)):
#         print(i, ini_param)
#         for j, param in enumerate(ini_param):
#             test_param = ini_param.copy()
#             test_param[j] = 0.5*param
#             if not pandas.isna(log_posterior(test_param)):
#                 print("parameter "+str(j-len(df["ID"].unique()))+" too large: "+str(param))
#             test_param[j] = 1.5*param
#             if not pandas.isna(log_posterior(test_param)):
#                 print("parameter "+str(j-len(df["ID"].unique()))+" too small: "+str(param))
#         optimisation.set_n_runs(n_runs)
#         i=0
#     else:
#         i+=1
#     if loop_number>50:
#         print('number of loops exceeded max')
#         break
#     loop_number += 1
optimisation.set_n_runs(n_runs)
optimisation.set_initial_point([1], [start_point])
optimisation.set_initial_point(range(2, 1+n_runs), initial_params[:n_runs-1, n_ids_data*len(model_mixed_params):], initial_params[:n_runs-1, :n_ids_data*len(model_mixed_params)])

In [ ]:
for i, param in enumerate(problem.get_parameter_names()):
    print(param, initial_params[:n_runs-1, n_ids_data*len(model_mixed_params)+i])

In [ ]:
optimisation.set_optimiser(pints.CMAES)
optimisation.set_transform(transformation)
optimisation.set_parallel_evaluation(True)
result = optimisation.run(show_run_progress_bar=False, log_to_screen=False)

# Show summary of optimisation
print('Log-Posterior Value: \t'+str(result['Score'].unique()))
time = np.asarray(result['Time'].unique())
print('Time Taken: \t'+str((time/60).astype(int))+" minutes, "+str((time%60).astype(int))+" seconds, ")


In [ ]:
parameters = np.empty((n_runs, len(start_point)))
opt_ind_params = np.empty((n_runs, len(result.dropna(subset=['ID'])['ID'].unique())*len(model_mixed_params)))
column_names = ['Parameter']
for run in range(1,n_runs+1):
    column_names.append('Run '+str(run))
    run_result = result.loc[result['Run']==run]
    # print(run, run_result.dropna(subset=['ID']))
    parameters[run-1] = run_result.loc[run_result['ID'].isnull()]['Estimate'].values
    opt_ind_params[run-1] = run_result.dropna(subset=['ID'])['Estimate'].values

column_names = column_names + ['Mean', 'True']
summary_data = np.concatenate((
    [np.concatenate((np.delete(
        pop_param_names,
        [x[1]+np.count_nonzero(np.asarray(list(model_mixed_params.values()))<x[1]) 
        for x in model_fixed_params.values()]
    ), ["Log-posterior"]))],
    np.concatenate((parameters, np.transpose([result['Score'].unique()])), axis=1),
    [np.concatenate((np.mean(parameters, axis=0), [np.NaN] ))],# log_posterior(np.mean(parameters, axis=0))))],
    [np.concatenate((np.delete(
        PKPD_real_pop_params,
        [x[1]+np.count_nonzero(np.asarray(list(model_mixed_params.values()))<x[1]) 
        for x in model_fixed_params.values()]
    ), [np.NaN] ))]# log_posterior(np.delete(PKPD_actual_pop_params, [4, 5, -2]))))]
), axis=0)
summary_df = pandas.DataFrame(summary_data.transpose(), columns = column_names)


summary_df.to_csv(PD_data_file+drug+PD_pop_model_file+fixed_file+"_opt_pop.csv", index=False)
np.save(PD_data_file+drug+PD_pop_model_file+fixed_file+"_opt_ind.npy", opt_ind_params)


In [ ]:
opt_ind_params = np.load(PD_data_file+drug+"/"+PD_pop_model_file+fixed_file+"_opt_ind.npy")
summary_df = pandas.read_csv(PD_data_file+drug+"/"+PD_pop_model_file+fixed_file+"_opt_pop.csv")

print('Result:')
table_df = summary_df
table_df = table_df.set_index('Parameter').transpose()
n_runs = len(table_df)-2
for param, pos in model_mixed_params.items():
    col = pop_param_names[PK_param_numbers[param]np.asarray(list(model_mixed_params.values()))<pos)]
    param_typ = np.asarray(table_df[col])
    param_typ[:-1] = np.exp(param_typ[:-1])
    table_df[col] = param_typ
rounding = dict(zip(table_df.columns, [3]*13))
rounding["Log-posterior"] = 2
table_df = table_df.round(rounding)
fig =  ff.create_table(table_df, index=True)
fig.update_layout(
    width=900,
    height=250,
)
fig.write_image(PD_image_file+"/"+PD_pop_model_file+fixed_file+"_opt_table.svg")
fig.show()

In [ ]:
from Code.Plot import Plot_Models

log_posterior_values = summary_df.loc["Log-posterior"].drop(["Mean", "True"])
log_posterior_values = np.asarray(log_posterior_values)

converged = log_posterior_values>=(np.max(log_posterior_values)+np.log(0.02))
summary_reduced = summary_df.copy()
summary_reduced = summary_reduced.iloc[:, :-2]
summary_reduced = summary_reduced.loc[:, np.logical_not(converged)]
mean_of_converged = summary_df.iloc[:, :-2].loc[:, converged].mean(axis=1)
summary_reduced["Mean of converged"] = mean_of_converged
summary_reduced["True"] = summary_df["True"]

table = summary_reduced.transpose()
i_mix = 0
n_mix = len([n for n in model_mixed_params if n in PK_params])
ind_params = np.empty((n_ids_data, len(summary_reduced) - 1 - n_mix))
for param, pos in PK_param_numbers.items():
    if param in model_mixed_params:
        col = pop_param_names[pos + i_mix]
        param_typ = np.asarray(table[col])
        param_typ[:-1] = np.exp(param_typ[:-1])
        table[col] = param_typ

        ind_params[:, pos] = opt_ind_params[converged, i_mix::n_mix].mean(axis=0)
        i_mix += 1
    else:
        ind_params[:, pos] = summary_reduced["Mean of converged"].iat[pos + i_mix]
rounding = dict(zip(table.columns, [3]*6))
rounding["Log-posterior"] = 2
table = table.round(rounding)
fig = ff.create_table(table, index=True)
fig.update_layout(
    width=500,
)
fig.write_image(PD_image_file+'/'+PD_pop_model_file+"_opt_table.svg")
fig.show()


plot = Plot_Models(
    pop_model=population_model,
    error_models=noise_model,
    mech_model=PK_model,
    data=df
)
# summary_df = summary_df.set_index("Parameter")
# fig = plot.plot_over_time(summary_reduced["Mean of converged"], ind_params=ind_params, show_data=True, doses=None, title="MLP Prediction")
# fig.write_image(PD_image_file+"/"+PD_pop_model_file+"opt_graph.svg")
# fig.show()


plot_pop_params = {"Mean of converged": summary_reduced["Mean of converged"]}
plot_ind_params = {"Mean of converged": ind_params}
for i_run, run in enumerate(summary_reduced.columns[:-2]):
    run_pop_params = summary_reduced[run]
    plot_pop_params["Failed " + run] = run_pop_params[:-1]
    ind_params = np.empty((n_ids_data, len(summary_reduced) - 1 - n_mix))
    i_mix = 0
    for param, pos in PK_param_numbers.items():
        if param in model_mixed_params:
            ind_params[:, pos] = opt_ind_params[np.logical_not(converged), i_mix::n_mix][i_run]
            i_mix += 1
        else:
            ind_params[:, pos] = run_pop_params[pos + i_mix]
    plot_ind_params["Failed " + run] = ind_params

fig = plot.plot_over_time(plot_pop_params, ind_params=plot_ind_params, show_data=True, doses=None, title="MLP Prediction", highlight_first=True)
fig.write_image(PD_image_file+"/"+PD_pop_model_file+"PD_opt_graph.svg")
fig.show()

In [ ]:
# Visualise the results
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from math import floor

n_times = 1000
n_ids = 1000


PK_time_span = df.loc[df["Observable"]=="PK_central.drug_concentration"]["Time"].max()
PD_time_span = df.loc[df["Observable"]=="circulating.R"]["Time"].max()
more_PK_times = np.linspace(
    start=0, stop=1.01*(PK_time_span-dose_time), num=n_times
)
more_PD_times = np.linspace(
    start=-dose_time, stop=1.01*(PD_time_span-dose_time), num=n_times
)

# Plot the PK Observations
fig1 = make_subplots(rows=4, cols=3, shared_xaxes=True, shared_yaxes=True)
fig2 = make_subplots(rows=4, cols=3, shared_xaxes=True, shared_yaxes=True)

non_pop_parameters = np.insert(PKPD_real_params[1], 6, 0.0)

for run in range(0, n_runs):
    row = floor(run/3)+1
    col = run%3+1
    print(row, col)
    inferred_pop_params = np.asarray(summary_df['Run '+str(run+1)])[:-1]
    avg_inferred = inferred_pop_params
    for x in all_params:
        if x in model_fixed_params:
            avg_inferred = np.insert(
                avg_inferred,
                model_fixed_params[x][1],
                non_pop_parameters[model_fixed_params[x][1]]
            )
        elif x in model_mixed_params:
            avg_inferred = np.delete(avg_inferred, model_mixed_params[x]+1) # log V_c std.
            avg_inferred[model_mixed_params[x]] = np.exp(avg_inferred[model_mixed_params[x]]) # V_c mean

    sim_ind_params = population_model.sample(
        parameters=inferred_pop_params,
        n_samples=n_ids
    )
    # Plot the population model for each dose
    ind = 0
    for i, group in enumerate(dose_groups):
        # Simulate and Plot from the population maximum likelihood estimation
        amt = dose_amts[i]
        PKPD_model.set_dosing_regimen(amt, start=dose_time, period=0)
        more_PK_values = PKPD_model.simulate(
            avg_inferred[:-2],
            more_PK_times+dose_time
        )[0]
        fig1.add_trace(
            go.Scatter(
                x=more_PK_times,
                y=more_PK_values,
                mode='lines',
                line=dict(color=ind_colours[i, int(n_per_dose[group]/2)], dash='dash'),
                name='Typical simulation',
                legendgroup= group,
                # legendgrouptitle = {'text': 'Dose '+str(group)+' '+dose_unit},
                showlegend=False
            ),
            row=row,
            col=col
        )
        more_PD_values = PKPD_model.simulate(avg_inferred[:-2], more_PD_times)[1]
        fig2.add_trace(
            go.Scatter(
                x=more_PD_times,
                y=more_PD_values,
                mode='lines',
                line=dict(color=ind_colours[i, int(n_per_dose[group]/2)], dash='dash'),
                name='Typical simulation',
                legendgroup= group,
                legendgrouptitle = {'text': 'Dose '+str(group)+' '+dose_unit},
                showlegend=False
            ),
            row=row,
            col=col
        )
        for j, patient in enumerate(df.loc[df['Dose group']==group]['ID'].unique()):
            PKPD_model.set_dosing_regimen(
                df.loc[(df['ID']==patient) & df['Observable'].isna()]['Dose'].iat[0], 
                start=dose_time,
                period=0
            )
            ind_real_params = avg_inferred.copy()
            for i_mix, pos in enumerate(model_mixed_params.values()):
                ind_real_params[pos] = opt_ind_params[run, ind*len(model_mixed_params) + i_mix]
            if group!=0.0:
                individual_PK_values = PKPD_model.simulate(
                    ind_real_params[:-2],
                    more_PK_times+dose_time
                )[0]
                fig1.add_trace(
                    go.Scatter(
                        x=more_PK_times,
                        y=individual_PK_values,
                        name='Individual simulation',
                        legendgroup = group,
                        showlegend= False,
                        mode="lines",
                        line=go.scatter.Line(color=ind_colours[i, j], width=0.5),
                        opacity=0.5,
                    ),
                    row=row,
                    col=col
                )
                x_data = df_PK_graph.loc[df_PK_graph['ID']==patient][x_label]-dose_time
                y_data = df_PK_graph.loc[df_PK_graph['ID']==patient][PK_y_label]
                fig1.add_trace(go.Scatter(
                    x=x_data,
                    y=y_data,
                    mode='markers',
                    marker_color=ind_colours[i, j],
                    name='Data',
                    legendgroup = group,
                    showlegend = False,
                ),row=row, col=col)
            individual_PD_values = PKPD_model.simulate(
                ind_real_params[:-2],
                more_PD_times+dose_time
            )[1]
            fig2.add_trace(
                go.Scatter(
                    x=more_PD_times,
                    y=individual_PD_values,
                    name='Individual simulation',
                    legendgroup = group,
                    showlegend= False,
                    mode="lines",
                    line=go.scatter.Line(color=ind_colours[i, j], width=0.5),
                    opacity=0.5,
                ),
                row=row,
                col=col
            )
            x_data = df_PD_graph.loc[df_PD_graph['ID']==patient][x_label]-dose_time
            y_data = df_PD_graph.loc[df_PD_graph['ID']==patient][PD_y_label]
            fig2.add_trace(
                go.Scatter(
                    x=x_data,
                    y=y_data,
                    mode='markers',
                    marker_color=ind_colours[i, j],
                    name='Data',
                    legendgroup = group,
                    showlegend = False,
                ),
                row=row,
                col=col
            )
            ind+=1
    
        PKPD_model.set_dosing_regimen(amt, start=dose_time, period=0)
        if group!=0.0:
            # Find 5th to 95th percentile of population distribution
            PK_simulations = np.empty(shape=(n_ids, n_times))
            PD_simulations = np.empty(shape=(n_ids, n_times))
            for idd, patient_parameters in enumerate(sim_ind_params):
                ind_param = patient_parameters.copy()
                for x in model_fixed_params.values():
                    ind_param = np.insert(
                        ind_param,
                        x[1],
                        non_pop_parameters[x[1]]
                    )
                # The solver has difficulties when V_c is too small (creates infinite drug concentration)
                if ind_param[2]<1e-10:
                    # PK_simulations[idd] = [np.inf]*len(more_PK_times)
                    # PD_simulations[idd] = [ind_param[2]]*np.count_nonzero(more_PD_times<0)+[0]*np.count_nonzero(more_PD_times>=0)
                    ind_param[2]=1e-10
                ind_sim = PKPD_model.simulate(
                    ind_param[:-2],
                    more_PK_times+dose_time
                )[0]
                PK_simulations[idd] = PK_noise_model.sample(
                    ind_param[-2:-1], ind_sim)[:, 0]
                ind_sim = PKPD_model.simulate(
                    ind_param[:-2],
                    more_PD_times + dose_time
                )[1]
                PD_simulations[idd] = PD_noise_model.sample(
                    ind_param[-1:], ind_sim)[:, 0]
            
            # Plot the variability
            fifth = np.percentile(PK_simulations, q=5, axis=0)
            ninety_fifth = np.percentile(PK_simulations, q=95, axis=0)
            fig1.add_trace(
                go.Scatter(
                    x=np.hstack([more_PK_times, more_PK_times[::-1]]),
                    y=np.hstack([fifth, ninety_fifth[::-1]]),
                    line=dict(
                        width=0,
                        color=ind_colours[i, int(n_per_dose[group]/2)]
                    ),
                    fill='toself',
                    name='Population model',
                    text=r"90% bulk probability",
                    hoverinfo='text',
                    legendgroup= group,
                    showlegend=False
                ),
                row=row,
                col=col
            )
            
        # Plot the variability
        fifth = np.percentile(PD_simulations, q=5, axis=0)
        ninety_fifth = np.percentile(PD_simulations, q=95, axis=0)
        fig2.add_trace(
            go.Scatter(
                x=np.hstack([more_PD_times, more_PD_times[::-1]]),
                y=np.hstack([fifth, ninety_fifth[::-1]]),
                line=dict(
                    width=0,
                    color=ind_colours[i, int(n_per_dose[group]/2)]
                ),
                fill='toself',
                name='Population model',
                text=r"90% bulk probability",
                hoverinfo='text',
                legendgroup= group,
                showlegend=False
            ),
            row=row,
            col=col
        )
    if run == 0:
        axis = ''
    else:
        axis = str(run+1)
    if col == 1:
        fig1['layout']['yaxis'+axis].update(title_text = PK_y_label, range=[np.log10(np.min(df_PK_graph[PK_y_label])/1.6), np.log10(1.6*np.max(df_PK_graph[PK_y_label]))])
        fig2['layout']['yaxis'+axis].update(title_text = PD_y_label, range=[np.min(df_PD_graph[PD_y_label])*0.9, 1.1*np.max(df_PD_graph[PD_y_label])])

    fig1['layout']['yaxis'+axis].update(type="log", minor=dict(dtick='D1', showgrid=True), dtick = 1)
    fig1['layout']['xaxis'+axis].update(dtick = 1, title_text = x_label)
    fig2['layout']['yaxis'+axis].update(dtick = 250)
    fig2['layout']['xaxis'+axis].update(dtick = 72, minor=dict(dtick=24, showgrid=True), title_text = x_label)

fig1.update_layout(
    width=1100,
    height=750,
    template='plotly_white'
)
fig2.update_layout(
    width=1100,
    height=750,
    template='plotly_white'
)

fig1.write_image(PD_image_file + "/" + PD_pop_model_file+fixed_file+"_PK_opt_graph.svg")
fig2.write_image(PD_image_file + "/" + PD_pop_model_file+fixed_file+"_PD_opt_graph.svg")
fig1.show()
fig2.show()

##### Comparison of fixed Parameters

In [ ]:
fixed_scenarios = [["K_cl", "K_1", "V_1", "sigma_PK"], ["K_1", "V_1", "sigma_PK"], ["V_1", "sigma_PK"], ["sigma_PK"], []]
# log_posterior = problem.get_log_posterior()

In [ ]:
# inferred_pop_params = pandas.read_csv(PD_data_file+drug+"/opt_pop_results"+fixed_param["\sigma_{PK}"]+".csv")
# inferred_pop_params = inferred_pop_params.set_index("Parameter")
# log_posterior_values = inferred_pop_params.loc["Log-posterior"].drop(["Mean", "True"])
# log_posterior_values = np.asarray(log_posterior_values)
# converged = log_posterior_values>=(np.max(log_posterior_values)+np.log(0.02))
# table = inferred_pop_params.iloc[:, :10]
# table = table.loc[:, np.logical_not(converged)]
# mean_of_converged = inferred_pop_params.iloc[:, :10].loc[:, converged].mean(axis=1)
# table["Mean of converged"] = mean_of_converged
# table["True"] = inferred_pop_params["True"]
# inferred_pop_params

In [ ]:
table_of_pop_params = {}

for scenario in fixed_scenarios:
    fixed_file_compare = ""
    if len(scenario)>0:
        fixed_file_compare += "_fixed"
        for param in scenario:
            fixed_file_compare += "_"+param
    
    inferred_pop_params = pandas.read_csv(PD_data_file+drug+PD_pop_model_file+fixed_file+"_opt_pop.csv")
    opt_ind_params = np.load(PD_data_file+drug+PD_pop_model_file+fixed_file+"_opt_ind.npy")
    inferred_pop_params = inferred_pop_params.set_index("Parameter")
    try:
        log_posterior_values = inferred_pop_params.loc["Log-posterior"].drop(["Mean", "True"])
        log_posterior_values = np.asarray(log_posterior_values)
    except KeyError:
        print("calculating posterior values for model with "+np.char.join(", ", scenario)+" PK parameters fixed")
        log_posterior_values = []
        for i in range(1, len(inferred_pop_params.columns)-1):
            params = inferred_pop_params['Run ' +str(i)]
            params = np.concatenate((opt_ind_params[i-1], params))
            log_posterior_values.append(log_posterior(params))
        inferred_pop_params.loc["Log-posterior"] = np.concatenate((log_posterior_values, [np.NaN, np.NaN]))
        inferred_pop_params = inferred_pop_params.reset_index(names='Parameter')
        inferred_pop_params.to_csv(PD_data_file+drug+PD_pop_model_file+fixed_file+"_opt_pop.csv", index=False)
        inferred_pop_params = inferred_pop_params.set_index("Parameter")

    converged = log_posterior_values>=(np.max(log_posterior_values)+np.log(0.02))
    table = inferred_pop_params.iloc[:, :-2]
    table = table.loc[:, np.logical_not(converged)]
    mean_of_converged = inferred_pop_params.iloc[:, :10].loc[:, converged].mean(axis=1)
    table["Mean of converged"] = mean_of_converged
    table["True"] = inferred_pop_params["True"]

    table_of_pop_params[scenario] = table


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

n_times = 1000
n_ids = 1000
PD_time_span = df.loc[df["Observable"]=="circulating.R"]["Time"].max()
more_PD_times = np.linspace(
    start=-dose_time, stop=1.01*(PD_time_span-dose_time), num=n_times
)

non_pop_parameters = np.insert(PKPD_real_params[1], 6, 0.0)

for scenario in fixed_scenarios:
    fixed_file_compare = ""
    if len(scenario)>0:
        fixed_file_compare += "_fixed"
        for param in scenario:
            fixed_file_compare += "_"+param

    table = table_of_pop_params[scenario]
    fig = make_subplots(rows=1, cols=len(table.columns)+2,
        shared_yaxes=True,
        vertical_spacing=0.01,
        horizontal_spacing=0.01,
        specs=[[{"type": "table", "colspan": 3}, None, None]+[{"type": "scatter"}]*(len(table.columns)-1)])

    # First subplot is table
    display_table = table.copy()
    for param, pos in model_mixed_params.items():
        param_typ = np.asarray(display_table[PKPD_real_params[pos]])
        param_typ[:-1] = np.exp(param_typ[:-1])
        display_table[PKPD_real_params[pos]] = param_typ
    rounding = dict(zip(display_table.columns, [3]*13))
    rounding["Log-posterior"] = 2
    display_table = display_table.rename(index = {'Log-posterior': 'Log-post'}, columns = {'Mean of converged': 'Converged'})
    display_table = display_table.round(rounding) # Round the values in the table for viewability
    display_table = display_table.reset_index(names='Parameter')

    fig.add_trace(go.Table(
            header=dict(values=list(display_table.columns)),
            cells=dict(values=display_table.to_numpy().transpose()),
            columnwidth=[2]+[1]*len(table.columns),
        ), row=1, col=1)
    
    # Rest of the Subplots are for graphing outliers
    for run, col in enumerate(table.columns):
        inferred_pop_params = np.asarray(table[col])[:-1]
        avg_inferred = inferred_pop_params
        for x in all_params:
            if x in scenario:
                avg_inferred = np.insert(
                    avg_inferred,
                    PKPD_param_numbers[x],
                    non_pop_parameters[PKPD_param_numbers[x]]
                )
            elif x in model_mixed_params:
                avg_inferred = np.delete(avg_inferred, model_mixed_params[x]+1) # log V_c std.
                avg_inferred[model_mixed_params[x]] = np.exp(avg_inferred[model_mixed_params[x]]) # V_c mean

        sim_ind_params = population_model.sample(
            parameters=inferred_pop_params,
            n_samples=n_ids
        )
        # Plot the population model for each dose
        ind = 0
        for i, group in enumerate(dose_groups):
            # Simulate and Plot from the population maximum likelihood estimation
            amt = dose_amts[i]
            PKPD_model.set_dosing_regimen(amt, start=dose_time, period=0)
            more_PD_values = PKPD_model.simulate(avg_inferred[:-2], more_PD_times)[1]
            fig.add_trace(
                go.Scatter(
                    x=more_PD_times,
                    y=more_PD_values,
                    mode='lines',
                    line=dict(color=ind_colours[i, int(n_per_dose[group]/2)], dash='dash'),
                    name='Typical simulation',
                    legendgroup= group,
                    legendgrouptitle = {'text': 'Dose '+str(group)+' '+dose_unit},
                    showlegend=False
                ),
                row=1,
                col=run+3
            )
            for j, patient in enumerate(df.loc[df['Dose group']==group]['ID'].unique()):
                PKPD_model.set_dosing_regimen(
                    df.loc[(df['ID']==patient) & df['Observable'].isna()]['Dose'].iat[0], 
                    start=dose_time,
                    period=0
                )
                # individual_parameters = avg_inferred.copy()
                # individual_parameters[2] = opt_ind_params[run, ind]
                # individual_PD_values = PKPD_model.simulate(
                #     individual_parameters[:-2],
                #     more_PD_times+dose_time
                # )[1]
                # fig.add_trace(
                #     go.Scatter(
                #         x=more_PD_times,
                #         y=individual_PD_values,
                #         name='Individual simulation',
                #         legendgroup = group,
                #         showlegend= False,
                #         mode="lines",
                #         line=go.scatter.Line(color=ind_colour_selection[i, j], width=0.5),
                #         opacity=0.5,
                #     ),
                #     row=1,
                #     col=run+4
                # )
                x_data = df_PD_graph.loc[df_PD_graph['ID']==patient][x_label]-dose_time
                y_data = df_PD_graph.loc[df_PD_graph['ID']==patient][PD_y_label]
                fig.add_trace(
                    go.Scatter(
                        x=x_data,
                        y=y_data,
                        mode='markers',
                        marker_color=ind_colours[i, j],
                        name='Data',
                        legendgroup = group,
                        showlegend = False,
                    ),
                    row=1,
                    col=run+4
                )
                ind+=1
        
            PKPD_model.set_dosing_regimen(amt, start=dose_time, period=0)
            if group!=0.0:
                # Find 5th to 95th percentile of population distribution
                PD_simulations = np.empty(shape=(n_ids, n_times))
                for idd, patient_parameters in enumerate(sim_ind_params):
                    ind_param = patient_parameters.copy()
                    for x in scenario:
                        ind_param = np.insert(
                            ind_param,
                            PKPD_param_numbers[x],
                            non_pop_parameters[PKPD_param_numbers[x]]
                        )
                    ind_sim = PKPD_model.simulate(
                        ind_param[:-2],
                        more_PD_times + dose_time
                    )[1]
                    PD_simulations[idd] = PD_noise_model.sample(
                        ind_param[-1:], ind_sim)[:, 0]
                
            # Plot the variability
            fifth = np.percentile(PD_simulations, q=5, axis=0)
            ninety_fifth = np.percentile(PD_simulations, q=95, axis=0)
            fig.add_trace(
                go.Scatter(
                    x=np.hstack([more_PD_times, more_PD_times[::-1]]),
                    y=np.hstack([fifth, ninety_fifth[::-1]]),
                    line=dict(
                        width=0,
                        color=ind_colours[i, int(n_per_dose[group]/2)]
                    ),
                    fill='toself',
                    name='Population model',
                    text=r"90% bulk probability",
                    hoverinfo='text',
                    legendgroup= group,
                    showlegend=False
                ),
                row=1,
                col=run+4
            )
        # if run == 0:
        #     axis = ''
        else:
            axis = str(run+4)
        fig2['layout']['yaxis'+axis].update(dtick = 250, title_text = PD_y_label)
        fig2['layout']['xaxis'+axis].update(dtick = 72, minor=dict(dtick=24, showgrid=True), title_text = x_label)

        fig.update_layout(
            width=1100,
            height=750,
            template='plotly_white'
        )

        fig.write_image(PD_image_file+"/"+PD_pop_model_file+fixed_file+"_compare_opt_graph.svg")
        fig.show()


#### Bayesian Inference

In [ ]:
# Further adjustables
num_samples = 1500  # number of wanted final samples from all runs
max_runs = 5
r_hat_criteria = 1.1
log_posterior = problem.get_log_posterior()

In [ ]:
# from Code.Inference import SamplingController
# import xarray as xr
# sampler = SamplingController(log_posterior)
# sampler.set_n_runs(max_runs)

# previous_samples = xr.open_dataset(PD_data_file+drug+PD_pop_model_file+fixed_file+"_samples.nc")
# sampler.reset_chains()
# sampler.add_samples(previous_samples)
# previous_samples.close()

# sampler.set_sampler(pints.NoUTurnMCMC)
# sampler.set_stop_criterion(max_iterations=int(num_samples/max_runs)+200, r_hat=r_hat_criteria)
# sampler.set_transform(transformation)
# posterior_samples = sampler.run(
#     n_iterations=int(num_samples/max_runs)+200,
#     log_to_screen=True,
#     reset=False
# )
# posterior_samples.to_netcdf(PD_data_file+drug+PD_pop_model_file+fixed_file+"_samples.nc")

In [ ]:
from Code.Inference import SamplingController
sampler = SamplingController(log_posterior)
sampler.set_n_runs(max_runs)

# ini_param = log_posterior.sample_initial_parameters(n_samples=max_runs*100)
# n_successful = 0
# for i in range(0, max_runs*100):
#     lp = log_posterior(ini_param[i])
#     if not(pandas.isna(lp) or (lp <= -1e8)):
#         n_successful += 1
#         top_param =ini_param[i, len(df['ID'].unique()):]
#         bottom_param = ini_param[i, :len(df['ID'].unique())]
#         sampler.set_initial_point(n_successful, top_param, bottom_param)
#         print(lp)
#     if n_successful==max_runs:
#         print("Successful Sample after", i, "Generations")
#         break
#     if i==max_runs*100-1:
#         print(
#              "Unsuccessful Sample, only",
#              n_successful,
#              "parameter sets accquired"
#         )

inferred_pop_params = pandas.read_csv(
    PD_data_file + drug
    + PD_pop_model_file + fixed_file
    + "_opt_pop.csv"
)
opt_ind_params = np.load(
    PD_data_file + drug
    + PD_pop_model_file + fixed_file
    + "_opt_ind.npy"
)

inferred_pop_params = inferred_pop_params.set_index("Parameter")
try:
    log_posterior_values = inferred_pop_params.loc["Log-posterior"].drop(["Mean", "True"])
    log_posterior_values = np.asarray(log_posterior_values)
except KeyError:
    print(
        "calculating posterior values for model with " +
        np.char.join(", ", scenario) +
        " PK parameters fixed"
    )
    log_posterior_values = []
    for i in range(1, len(inferred_pop_params.columns)-1):
        params = inferred_pop_params['Run '+str(i)]
        params = np.concatenate((opt_ind_params[i-1], params))
        log_posterior_values.append(log_posterior(params))
    inferred_pop_params.loc["Log-posterior"] = np.concatenate((
        log_posterior_values, [np.NaN, np.NaN]
    ))
    inferred_pop_params = inferred_pop_params.reset_index(names='Parameter')
    inferred_pop_params.to_csv((
        PD_data_file + drug +
        PD_pop_model_file + fixed_file +
        "_opt_pop.csv"
    ), index=False)
    inferred_pop_params = inferred_pop_params.set_index("Parameter")

converged = log_posterior_values >= (np.max(log_posterior_values)+np.log(0.02))
n_runs = np.count_nonzero(converged)

if n_runs < 3:
    print(
        "Not enough consistent results from the Maximum"+
        "Likelihood Estimation to start Sampling"
    )
else:
    if n_runs >= max_runs:
        n_runs = max_runs
    sampler.set_n_runs(n_runs)
    opt_params = (
        inferred_pop_params.drop(columns=['Mean', 'True']).values[:, converged]
    )[:-1, :n_runs]
    ind_params = (opt_ind_params[converged])[:n_runs]
    sampler.set_initial_point(
        range(1, n_runs+1), opt_params.transpose(), ind_params
    )
    for i, param in enumerate(population_model.get_parameter_names()):
        print(param, opt_params[i])

In [ ]:
cov_0 = None
sampler.set_sampler(pints.NoUTurnMCMC)
sampler.set_stop_criterion(
    max_iterations=int(num_samples/n_runs)+100,
    r_hat=r_hat_criteria
)
sampler.set_transform(transformation)
posterior_samples = sampler.run(
    n_iterations=int(num_samples/n_runs)+100, sigma0=cov_0
)
posterior_samples.to_netcdf(
    PD_data_file+drug+PD_pop_model_file+fixed_file+"_samples.nc"
)

In [ ]:
import xarray as xr
posterior_samples = xr.open_dataset(
    PD_data_file+drug+PD_pop_model_file+fixed_file+"_samples.nc"
).load()
n_runs=len(posterior_samples.chain)

In [ ]:
import arviz as az
import matplotlib.pyplot as plt

az.style.use(["arviz-white",  "arviz-viridish"])

lines = list(zip(*(
    population_model.get_parameter_names(),
    [{}]*len(population_model.get_parameter_names()),
    np.delete(PKPD_real_pop_params, [4, 5, -2]))))
actual_ind_Vc = np.load(PD_data_file + drug+"/ind_Vc_param.npy")
lines = lines+[(
    "PK_central.V_c",
    dict([[
        "individual", df['ID'].unique()
    ]]),
    actual_ind_Vc
)]
# lines[1] = (
#     '',# population_model.get_parameter_names()[1],
#     {},
#     np.exp(PKPD_actual_pop_params[1])
# )
lines[2] = (
    population_model.get_parameter_names()[2],
    {},
    np.exp(PKPD_real_pop_params[2])
)

In [ ]:
def graph_tansform(params_to_trans):
    trans_param = params_to_trans
    try:
        trans_param['Log mean '+param] = np.exp(trans_param['Log mean '+param])
    except:
        pass
    return trans_param

# Discard warmup iterations
discard = max(len(posterior_samples.draw)-int(num_samples/n_runs), 1)
main_samples = posterior_samples.drop_isel(draw=slice(0, discard))

print("R-hat value:", az.rhat(main_samples))

for param in np.delete(PKPD_model.parameters(), [3, 4, 6]):
    az.plot_trace(
        main_samples,
        var_names=('.*'+param),
        filter_vars="regex",
        lines=lines,
        transform=graph_tansform
    )
    param_name = param.split('.')[-1]
    plt.savefig(
        PD_image_file + "/" +
        PD_pop_model_file + fixed_file +
        "MCMC_"+param_name+"_trace.svg"
    )
    plt.show()

az.plot_trace(main_samples, var_names=('.*Sigma'), filter_vars="regex", lines=lines)
plt.savefig(
    PD_image_file + "/" +
    PD_pop_model_file + fixed_file +
    "MCMC_noise_trace.svg"
)
plt.show()

In [ ]:
posterior_samples

In [ ]:
# Visualise the results
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from math import floor

n_times = 1000
n_ids = 100  # number of ids to generate per sample for determining percentiles
n_samples_per = 100 # number of samples to select for determining percentiles

PK_time_span = df.loc[df["Observable"]=="PK_central.drug_concentration"]["Time"].max()
PD_time_span = df.loc[df["Observable"]=="circulating.R"]["Time"].max()
more_PK_times = np.linspace(
    start=0, stop=1.01*(PK_time_span-dose_time), num=n_times
)
more_PD_times = np.linspace(
    start=-dose_time, stop=1.01*(PD_time_span-dose_time), num=n_times
)



# Plot the PK Observations
fig1 = make_subplots(rows=4, cols=3, shared_xaxes=True, shared_yaxes=True)
fig2 = make_subplots(rows=4, cols=3, shared_xaxes=True, shared_yaxes=True)

for run in range(0, n_runs):
    row = floor(run/3)+1
    col = run%3+1
    print(row, col)
    sampled_params = main_samples.isel(chain=[run])
    mean_samples = sampled_params.mean(dim="draw")
    avg_inferred = np.asarray(mean_samples.drop(['PK_central.V_c']).to_array()).flatten()
    # avg_inferred = np.insert(avg_inferred, 1, PK_opt_params[0]) # Clearance
    avg_inferred[2] = np.exp(avg_inferred[2]) # V_c mean
    avg_inferred = np.delete(avg_inferred, [3]) # log V_c std.
    avg_inferred = np.insert(avg_inferred, 3, PK_opt_params[3]) # Periferal Parameters
    avg_inferred = np.insert(avg_inferred, 4, PK_opt_params[4]) # Periferal Parameters
    avg_inferred = np.insert(avg_inferred, 6, 0) # Nonsense paremeter
    avg_inferred = np.insert(avg_inferred, -1, PK_opt_params[-1]) # PK Noise parameter

    
    # sim_ind_params = population_model.sample(
    #     parameters=inferred_pop_params,
    #     n_samples=n_ids
    # )
    
    per_ind_params = np.empty((0,7))
    per_samples = az.extract(sampled_params, num_samples=n_samples_per)
    for s in per_samples.coords['sample']:
        param = np.asarray(per_samples.drop(['PK_central.V_c']).sel(sample=s).to_array()).flatten()
        per_ind_params = np.concatenate((per_ind_params, population_model.sample(
            parameters=param,
            n_samples=n_ids
        )), axis = 0)

    # Plot the population model for each dose
    for i, group in enumerate(dose_groups):
        # Simulate and Plot from the population mean
        amt = dose_amts[i]
        PKPD_model.set_dosing_regimen(amt, start=dose_time, period=0)
        more_PK_values = PKPD_model.simulate(
            avg_inferred[:-2],
            more_PK_times+dose_time
        )[0]
        fig1.add_trace(
            go.Scatter(
                x=more_PK_times,
                y=more_PK_values,
                mode='lines',
                line=dict(color=ind_colours[i, int(n_ids_data/2)], dash='dash'),
                name='Typical simulation',
                legendgroup= group,
                # legendgrouptitle = {'text': 'Dose '+str(group)+' '+dose_unit},
                showlegend=False
            ),
            row=row,
            col=col
        )
        more_PD_values = PKPD_model.simulate(avg_inferred[:-2], more_PD_times)[1]
        fig2.add_trace(
            go.Scatter(
                x=more_PD_times,
                y=more_PD_values,
                mode='lines',
                line=dict(color=ind_colours[i, int(n_ids_data/2)], dash='dash'),
                name='Typical simulation',
                legendgroup= group,
                legendgrouptitle = {'text': 'Dose '+str(group)+' '+dose_unit},
                showlegend=False
            ),
            row=row,
            col=col
        )
        for j, patient in enumerate(df.loc[df['Dose group']==group]['ID'].unique()):
            PKPD_model.set_dosing_regimen(
                df.loc[(df['ID']==patient) & df['Observable'].isna()]['Dose'].iat[0], 
                start=dose_time,
                period=0
            )
            ind_real_params = avg_inferred.copy()
            ind_real_params[2] = mean_samples['PK_central.V_c'].sel(individual=str(patient)).item()
            if group!=0.0:
                individual_PK_values = PKPD_model.simulate(
                    ind_real_params[:-2],
                    more_PK_times+dose_time
                )[0]
                fig1.add_trace(
                    go.Scatter(
                        x=more_PK_times,
                        y=individual_PK_values,
                        name='Individual simulation',
                        legendgroup = group,
                        showlegend= False,
                        mode="lines",
                        line=go.scatter.Line(color=ind_colours[i, j], width=0.5),
                        opacity=0.5,
                    ),
                    row=row,
                    col=col
                )
                x_data = df_PK_graph.loc[df_PK_graph['ID']==patient][x_label]-dose_time
                y_data = df_PK_graph.loc[df_PK_graph['ID']==patient][PK_y_label]
                fig1.add_trace(go.Scatter(
                    x=x_data,
                    y=y_data,
                    mode='markers',
                    marker_color=ind_colours[i, j],
                    name='Data',
                    legendgroup = group,
                    showlegend = False,
                ),row=row, col=col)
            individual_PD_values = PKPD_model.simulate(
                ind_real_params[:-2],
                more_PD_times+dose_time
            )[1]
            fig2.add_trace(
                go.Scatter(
                    x=more_PD_times,
                    y=individual_PD_values,
                    name='Individual simulation',
                    legendgroup = group,
                    showlegend= False,
                    mode="lines",
                    line=go.scatter.Line(color=ind_colours[i, j], width=0.5),
                    opacity=0.5,
                ),
                row=row,
                col=col
            )
            x_data = df_PD_graph.loc[df_PD_graph['ID']==patient][x_label]-dose_time
            y_data = df_PD_graph.loc[df_PD_graph['ID']==patient][PD_y_label]
            fig2.add_trace(
                go.Scatter(
                    x=x_data,
                    y=y_data,
                    mode='markers',
                    marker_color=ind_colours[i, j],
                    name='Data',
                    legendgroup = group,
                    showlegend = False,
                ),
                row=row,
                col=col
            )
            
        PKPD_model.set_dosing_regimen(amt, start=dose_time, period=0)
        if group!=0.0:
            # Find 5th to 95th percentile of population distribution
            PK_simulations = np.empty(shape=(len(per_ind_params), n_times))
            PD_simulations = np.empty(shape=(len(per_ind_params), n_times))
            for idd, patient_parameters in enumerate(per_ind_params):
                # avg_inferred = np.insert(avg_inferred, 1, PK_opt_params[0]) # Clearance
                ind_param = patient_parameters.copy()
                ind_param = np.insert(ind_param, 3, PK_opt_params[3]) # Periferal Parameters
                ind_param = np.insert(ind_param, 4, PK_opt_params[4]) # Periferal Parameters
                ind_param = np.insert(ind_param, 6, 0) # Nonsense paremeter
                ind_param = np.insert(ind_param, -1, PK_opt_params[-1]) # PK Noise parameter
                ind_sim = PKPD_model.simulate(
                    ind_param[:-2],
                    more_PK_times+dose_time
                )[0]
                PK_simulations[idd] = PK_noise_model.sample(
                    ind_param[-2:-1], ind_sim)[:, 0]
                ind_sim = PKPD_model.simulate(
                    ind_param[:-2],
                    more_PD_times + dose_time
                )[1]
                PD_simulations[idd] = PD_noise_model.sample(
                    ind_param[-1:], ind_sim)[:, 0]
            
            # Plot the variability
            fifth = np.percentile(PK_simulations, q=5, axis=0)
            ninety_fifth = np.percentile(PK_simulations, q=95, axis=0)
            fig1.add_trace(
                go.Scatter(
                    x=np.hstack([more_PK_times, more_PK_times[::-1]]),
                    y=np.hstack([fifth, ninety_fifth[::-1]]),
                    line=dict(
                        width=0,
                        color=ind_colours[i, int(n_ids_data/2)]
                    ),
                    fill='toself',
                    name='Population model',
                    text=r"90% bulk probability",
                    hoverinfo='text',
                    legendgroup= group,
                    showlegend=False
                ),
                row=row,
                col=col
            )
            
        # Plot the variability
        fifth = np.percentile(PD_simulations, q=5, axis=0)
        ninety_fifth = np.percentile(PD_simulations, q=95, axis=0)
        fig2.add_trace(
            go.Scatter(
                x=np.hstack([more_PD_times, more_PD_times[::-1]]),
                y=np.hstack([fifth, ninety_fifth[::-1]]),
                line=dict(
                    width=0,
                    color=ind_colours[i, int(n_ids_data/2)]
                ),
                fill='toself',
                name='Population model',
                text=r"90% bulk probability",
                hoverinfo='text',
                legendgroup= group,
                showlegend=False
            ),
            row=row,
            col=col
        )
    if run == 0:
        axis = ''
    else:
        axis = str(run+1)
    fig1['layout']['yaxis'+axis].update(type="log", minor=dict(dtick='D1', showgrid=True), dtick = 1, title_text = PK_y_label)
    fig1['layout']['xaxis'+axis].update(dtick = 1, title_text = x_label)
    fig2['layout']['yaxis'+axis].update(dtick = 250, title_text = PD_y_label)
    fig2['layout']['xaxis'+axis].update(dtick = 72, minor=dict(dtick=24, showgrid=True), title_text = x_label)

fig1.update_layout(
    width=1100,
    height=750,
    template='plotly_white'
)
fig2.update_layout(
    width=1100,
    height=750,
    template='plotly_white'
)

fig1.write_image(PD_image_file+"/PK_MCMC_runs_graph_excl_Perif_PKnoise.svg")
fig2.write_image(PD_image_file+"/PD_MCMC_runs_graph_excl_Perif_PKnoise.svg")
fig1.show()
fig2.show()

In [ ]:
# Visualise the results
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from math import floor

n_times = 1000
n_samples_per = 100  # number of samples to select for determining percentiles
n_ids = 100  # number of ids to generate per sample for determining percentiles

PK_time_span = df.loc[df["Observable"] == "PK_central.drug_concentration"]["Time"].max()
PD_time_span = df.loc[df["Observable"] == "circulating.R"]["Time"].max()
more_PK_times = np.linspace(
    start=0, stop=1.01*(PK_time_span-dose_time), num=n_times
)
more_PD_times = np.linspace(
    start=-dose_time, stop=1.01*(PD_time_span-dose_time), num=n_times
)

# Plot the PK Observations
fig = make_subplots(rows=1, cols=2)

mean_samples = main_samples.mean(dim=["draw", "chain"])
avg_inferred = np.asarray(mean_samples.drop(['PK_central.V_c']).to_array()).flatten()
# avg_inferred = np.insert(avg_inferred, 1, PK_opt_params[0]) # Clearance
avg_inferred[2] = np.exp(avg_inferred[2])                    # V_c mean
avg_inferred = np.delete(avg_inferred, [3])                  # log V_c std.
avg_inferred = np.insert(avg_inferred, 3, PK_opt_params[3])  # Periferal Parameters
avg_inferred = np.insert(avg_inferred, 4, PK_opt_params[4])  # Periferal Parameters
avg_inferred = np.insert(avg_inferred, 6, 0)                 # Nonsense paremeter
avg_inferred = np.insert(avg_inferred, -1, PK_opt_params[-1])  # PK Noise parameter


# sim_ind_params = population_model.sample(
#     parameters=inferred_pop_params,
#     n_samples=n_ids
# )

per_ind_params = np.empty((0, 7))
per_samples = az.extract(main_samples, num_samples=n_samples_per)
for s in per_samples.coords['sample']:
    param = np.asarray(
        per_samples.drop(['PK_central.V_c']).sel(sample=s).to_array()
    ).flatten()
    per_ind_params = np.concatenate((per_ind_params, population_model.sample(
        parameters=param,
        n_samples=n_ids
    )), axis=0)

# Plot the population model for each dose
for i, group in enumerate(dose_groups):
    # Simulate and Plot from the population mean
    amt = dose_amts[i]
    PKPD_model.set_dosing_regimen(amt, start=dose_time, period=0)
    more_PK_values = PKPD_model.simulate(
        avg_inferred[:-2],
        more_PK_times+dose_time
    )[0]
    fig.add_trace(
        go.Scatter(
            x=more_PK_times,
            y=more_PK_values,
            mode='lines',
            line=dict(
                color=ind_colours[i, int(n_ids_data/2)],
                dash='dash'
            ),
            name='Typical simulation',
            legendgroup=group,
            # legendgrouptitle = {'text': 'Dose '+str(group)+' '+dose_unit},
            showlegend=False
        ),
        row=1,
        col=1
    )
    more_PD_values = PKPD_model.simulate(avg_inferred[:-2], more_PD_times)[1]
    fig.add_trace(
        go.Scatter(
            x=more_PD_times,
            y=more_PD_values,
            mode='lines',
            line=dict(
                color=ind_colours[i, int(n_ids_data/2)],
                dash='dash'
            ),
            name='Typical simulation',
            legendgroup=group,
            legendgrouptitle={'text': 'Dose '+str(group)+' '+dose_unit},
            showlegend=False
        ),
        row=1,
        col=2
    )
    for j, patient in enumerate(df.loc[df['Dose group'] == group]['ID'].unique()):
        ind_dose = df.loc[(df['ID']==patient) & df['Observable'].isna()]
        PKPD_model.set_dosing_regimen(
            ind_dose['Dose'].iat[0],
            start=dose_time,
            period=0
        )
        ind_real_params = avg_inferred.copy()
        ind_Vc = mean_samples['PK_central.V_c'].sel(individual=str(patient))
        ind_real_params[2] = ind_Vc.item()
        if group != 0.0:
            individual_PK_values = PKPD_model.simulate(
                ind_real_params[:-2],
                more_PK_times+dose_time
            )[0]
            fig.add_trace(
                go.Scatter(
                    x=more_PK_times,
                    y=individual_PK_values,
                    name='Individual simulation',
                    legendgroup=group,
                    showlegend=False,
                    mode="lines",
                    line=go.scatter.Line(
                        color=ind_colours[i, j],
                        width=0.5
                    ),
                    opacity=0.5,
                ),
                row=1,
                col=1
            )
            ind_data = df_PK_graph.loc[df_PK_graph['ID'] == patient]
            x_data = ind_data[x_label] - dose_time
            y_data = ind_data[PK_y_label]
            fig.add_trace(go.Scatter(
                x=x_data,
                y=y_data,
                mode='markers',
                marker_color=ind_colours[i, j],
                name='Data',
                legendgroup=group,
                showlegend=False,
            ), row=1, col=1)
        individual_PD_values = PKPD_model.simulate(
            ind_real_params[:-2],
            more_PD_times+dose_time
        )[1]
        fig.add_trace(
            go.Scatter(
                x=more_PD_times,
                y=individual_PD_values,
                name='Individual simulation',
                legendgroup=group,
                showlegend=False,
                mode="lines",
                line=go.scatter.Line(
                    color=ind_colours[i, j],
                    width=0.5
                ),
                opacity=0.5,
            ),
            row=1,
            col=2
        )
        x_data = df_PD_graph.loc[df_PD_graph['ID'] == patient][x_label] - dose_time
        y_data = df_PD_graph.loc[df_PD_graph['ID'] == patient][PD_y_label]
        fig.add_trace(
            go.Scatter(
                x=x_data,
                y=y_data,
                mode='markers',
                marker_color=ind_colours[i, j],
                name='Data',
                legendgroup=group,
                showlegend=False,
            ),
            row=1,
            col=2
        )

    PKPD_model.set_dosing_regimen(amt, start=dose_time, period=0)
    if group != 0.0:
        # Find 5th to 95th percentile of population distribution
        PK_simulations = np.empty(shape=(len(per_ind_params), n_times))
        PD_simulations = np.empty(shape=(len(per_ind_params), n_times))
        for idd, patient_parameters in enumerate(per_ind_params):
            # avg_inferred = np.insert(avg_inferred, 1, PK_opt_params[0])  # Clearance
            ind_param = patient_parameters.copy()
            ind_param = np.insert(ind_param, 3, PK_opt_params[3])  # Periferal Parameters
            ind_param = np.insert(ind_param, 4, PK_opt_params[4])  # Periferal Parameters
            ind_param = np.insert(ind_param, 6, 0)                 # Nonsense paremeter
            ind_param = np.insert(ind_param, -1, PK_opt_params[-1])  # PK Noise parameter
            ind_sim = PKPD_model.simulate(
                ind_param[:-2],
                more_PK_times+dose_time
            )[0]
            PK_simulations[idd] = PK_noise_model.sample(
                ind_param[-2:-1], ind_sim)[:, 0]
            ind_sim = PKPD_model.simulate(
                ind_param[:-2],
                more_PD_times + dose_time
            )[1]
            PD_simulations[idd] = PD_noise_model.sample(
                ind_param[-1:], ind_sim
            )[:, 0]

        # Plot the variability
        fifth = np.percentile(PK_simulations, q=5, axis=0)
        ninety_fifth = np.percentile(PK_simulations, q=95, axis=0)
        fig.add_trace(
            go.Scatter(
                x=np.hstack([more_PK_times, more_PK_times[::-1]]),
                y=np.hstack([fifth, ninety_fifth[::-1]]),
                line=dict(
                    width=0,
                    color=ind_colours[i, int(n_ids_data/2)]
                ),
                fill='toself',
                name='Population model',
                text=r"90% bulk probability",
                hoverinfo='text',
                legendgroup=group,
                showlegend=False
            ),
            row=1,
            col=1
        )

    # Plot the variability
    fifth = np.percentile(PD_simulations, q=5, axis=0)
    ninety_fifth = np.percentile(PD_simulations, q=95, axis=0)
    fig.add_trace(
        go.Scatter(
            x=np.hstack([more_PD_times, more_PD_times[::-1]]),
            y=np.hstack([fifth, ninety_fifth[::-1]]),
            line=dict(
                width=0,
                color=ind_colours[i, int(n_ids_data/2)]
            ),
            fill='toself',
            name='Population model',
            text=r"90% bulk probability",
            hoverinfo='text',
            legendgroup=group,
            showlegend=False
        ),
        row=1,
        col=2
    )
fig['layout']['yaxis'].update(
    type="log", minor=dict(dtick='D1', showgrid=True), dtick=1, title_text=PK_y_label
)
fig['layout']['xaxis'].update(dtick=1, title_text=x_label)
fig['layout']['yaxis2'].update(dtick=250, title_text=PD_y_label)
fig['layout']['xaxis2'].update(
    dtick=72, minor=dict(dtick=24, showgrid=True), title_text=x_label
)

fig.update_layout(
    width=1200,
    height=500,
    template='plotly_white'
)

fig.write_image(
    PD_image_file+"/PKPD_MCMC_tot_graph_excl_Perif_PKnoise.svg"
)
fig.show()

In [ ]:
posterior_samples.close()

#### Model Selection

In [ ]:
import xarray as xr
import arviz as az
log_likelihood = problem.get_log_posterior().get_log_likelihood()

In [ ]:
list(posterior_samples.data_vars)[0]

In [ ]:
class PointwiseLikelihood():

    def __init__(self, log_likelihood, data) -> None:
        self.log_likelihood = log_likelihood
        self.data = data
        self.ids = np.asarray(data["ID"].unique())
        self.n_obs_points = len(data.loc[(data["ID"]==self.ids[0]) & data["Value"].notna()])
        self.reset_pwll()

    def reset_pwll(self):
        self.pwlls = np.empty((0, len(self.ids), self.n_obs_points))

    def add_pwll(self, parameters):
        pwll = np.asarray(self.log_likelihood.compute_pointwise_ll(parameters, per_individual=False))
        pwll = pwll.reshape((1, len(self.ids), self.n_obs_points))
        self.pwlls = np.append(self.pwlls, pwll, axis=0)

    def create_pwlls(self, samples):
        self.reset_pwll()
        # add pointwise likelihood for each parameter sample
        for param in samples:
            self.add_pwll(param)
            # check if it agrees with loglikelihood
            sum_pw = np.sum(self.pwlls[-1])
            ll = self.log_likelihood(param)
            if np.isinf(sum_pw):
                if not np.isinf(sum_pw):
                    raise ValueError("pointwise log-likelihoods do not sum to log-likelihood, "+str(sum_pw)+"!="+str(ll))
            elif np.abs((sum_pw-ll)/ll) > 1e-1:
                    raise ValueError("pointwise log-likelihoods do not sum to log-likelihood, "+str(sum_pw)+"!="+str(ll))
        return self.pwlls
    
    def get_inference_data(self, samples):
        posterior = az.convert_to_inference_data(samples, group='posterior')
        
        obs_df = df.copy()
        obs_df = obs_df.dropna(subset=['Observable']).sort_values(['ID', 'Time', 'Observable'])
        obs_df["Points"] = (
            obs_df['Observable'].astype(str)+
            np.asarray([" - "]*len(obs_df)).astype(str) +
            obs_df['Time'].astype(str)
        )
        obs_df = obs_df[["ID", "Points", "Value"]]
        obs_df = obs_df.rename(columns={"ID": 'Individual'})
        obs_points = obs_df["Points"]
        obs_df = obs_df.set_index(['Individual', "Points"])
        observed = az.convert_to_inference_data(
            obs_df.to_xarray(),
            group='observed_data'
        )

        const_df = df.copy()
        const_df = const_df.dropna(subset=['Dose']).sort_values(['ID', 'Time'])
        const_df = const_df[['ID', "Time", "Dose group", "Duration", "Dose"]]
        const_df = const_df.rename(columns={'ID': 'Individual'})
        const_df = const_df.set_index(['Individual', "Time"])
        constant = az.convert_to_inference_data(
            obs_df.to_xarray(),
            group='constant_data'
        )
        ids = df["ID"].unique() # samples.coords["individual"]
        if isinstance(samples, (xr.Dataset, az.InferenceData)):
            pop_param_names = [var for var in posterior_samples.data_vars if " " in var]
            pop_samples = az.extract(posterior_samples, var_names=pop_param_names)
            sample_coords = pop_samples.coords["sample"]
            try:
                ind_samples = az.extract(posterior_samples, var_names=['~' + x for x in pop_param_names])
            except KeyError:
                ind_samples = np.empty((0, len(sample_coords)))
            if isinstance(ind_samples, xr.Dataset):
                flat_ind_samples = np.asarray(ind_samples[list(ind_samples.data_vars)[0]])
                for var in list(ind_samples.data_vars)[1:]:
                    flat_ind_samples = np.insert(flat_ind_samples, range(1, n_ids_data+1), ind_samples[var], axis=0)
                ind_samples = flat_ind_samples
            if isinstance(pop_samples, xr.Dataset):
                pop_samples = pop_samples.to_array()
            numpy_samples = np.concatenate((ind_samples, np.asarray(pop_samples))).transpose()
        else:
            numpy_samples = np.asarray(samples)
            sample_coords = np.arange(numpy_samples.shape[0])
        pwlls = np.asarray(np.split(self.create_pwlls(numpy_samples), len(samples.coords["draw"]), axis=0))
        pwlls = pwlls.transpose((1, 0, 2, 3))
        pointwise = {"pointwise log-likelihooods": pwlls}
        print(pointwise["pointwise log-likelihooods"].shape)
        coords = {"individual": ids, "Points": obs_points.unique()}
        dims = {"pointwise log-likelihooods": ["individual", "Points"]}
        likelihood = az.convert_to_inference_data(pointwise, coords=coords, dims=dims, group='log_likelihood')

        inference_data = az.concat(likelihood, posterior, observed, constant)
        return inference_data

In [ ]:
start_point = [V_c_approx]*(n_ids_data*3) + [
    0.5*PD_bounds[0,0]+0.5*PD_bounds[1,0],      # S
    cl_approx,                                 # Clearance
    np.log(V_c_approx),                         # Log mean of central volume
    0.1,                                        # Log std. of central volume
#     cl_approx*np.random.random(),             # Periferal compartment transfer
#     V_c_approx*np.random.random(),            # Periferal compartment volume
    R_0_approx,                                 # R_0
    1,                                          # gamma
    MTT_approx,                                 # MTT
#     0.1,                                      # PK Noise parameter
    0.1*R_0_approx*np.random.random(),          # PD Noise parameter
]

pw_ll = PointwiseLikelihood(log_likelihood, df)
pw_ll.add_pwll(start_point)
print(np.sum(pw_ll.pwlls), log_likelihood(start_point))

In [ ]:
posterior_samples = xr.open_dataset(PD_data_file+drug+"/"+PD_pop_model_file+fixed_file+"_samples.nc").load()
inference_data = pw_ll.get_inference_data(posterior_samples)
az.to_netcdf(inference_data, PD_data_file+drug+"/"+PD_pop_model_file+fixed_file+"_inference_obj.nc")
posterior_samples.close()
inference_data

In [ ]:
# Acquire Pointwise posterior samples
import xarray as xr
import arviz as az
model_types = ["fixed_effects", "pop_V_c", "pop_V_c_R_0"]
num_samples = 1500  # number of wanted final samples from all runs
compare_dict = {}


# For mixed effects model
for model_type in model_types:
    print('----------------------------------------------------------------------------------')
    print(model_type)
    inference_data = az.from_netcdf(PD_data_file+drug+"/"+model_type+fixed_file+"_inference_obj.nc")
    compare_dict[model_type] = inference_data

# compare_dict
az.compare(compare_dict)

In [ ]:
az.compare(compare_dict, ic="waic")

In [ ]:
posterior_samples.close()

In [ ]:
az.compare(compare_dict)